<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/Gold__1_%26_2_%26_3_20251112_1700.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIP

In [1]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl
!pip install xlsxwriter
!pip install mlxtend
!pip caas_jupyter_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 6.5 MB/s eta 0:00:00
ERROR: unknown command "caas_jupyter_tools"


# Ron Rules-QUICK CODE CARD (v2025.10.27-Refined)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Deliver Python code (inline, executable, CUT-ready section)
# - XHTML 1.0 Transitional; old-school friendly; Times New Roman body.
# - Use CUT markers; five # spacer lines follow the STOP marker.

In [ ]:
# ====== CUT START [1/1] CENTERLINE FIX — Update shared CSS + Upload =============================
# RON RULES — QUICK CODE CARD (v2025.11.06-CSS-Centerline-Fix)
# - Complete & runnable in Colab; ISO-8859-15 safe in source.
# - Purpose: ensure "Last updated …" (meta) is centered site-wide via shared stylesheet.
# - Actions: edit/insert CENTERLINE FIX block in dna_tree_styles.css and FTPS-upload to /partials/.

import os, re, socket, posixpath, traceback
from ftplib import FTP_TLS
from datetime import datetime

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR',  '')
    os.environ.setdefault('FTP_PORT', '21')

FTP_DIR   = (os.environ.get("FTP_DIR","") or "").strip()
LOCAL_CSS = "dna_tree_styles.css"
REMOTE_CSS = posixpath.join("partials", "dna_tree_styles.css")

# ---------- 1) Ensure local css exists (create if missing) ----------
if not os.path.exists(LOCAL_CSS):
    with open(LOCAL_CSS, "w", encoding="utf-8") as f:
        f.write("/* yates.one-name.net shared stylesheet */\n")
    print("[INFO] Created new local stylesheet:", os.path.abspath(LOCAL_CSS))
else:
    print("[OK] Using existing local stylesheet:", os.path.abspath(LOCAL_CSS))

# ---------- 2) Insert/replace CENTERLINE FIX block ----------
with open(LOCAL_CSS, "r", encoding="utf-8", errors="replace") as f:
    css = f.read()

start_tag = "/* === CENTERLINE FIX (do not remove) START === */"
end_tag   = "/* === CENTERLINE FIX (do not remove) END === */"
block_re  = re.compile(re.escape(start_tag) + r".*?" + re.escape(end_tag), flags=re.S)

center_block = f"""{start_tag}
:root {{
  /* nothing here yet; reserved for future tokens */
}}
/* Utility: hard center text, wins against generic .meta rules */
.centerline {{ text-align: center !important; }}

/* Meta display: keep centered across pages, even if other CSS sets .meta {{text-align:left}} */
.wrap .meta, .intro .meta, .updated, .updated.centerline {{
  text-align: center !important;
}}

/* Table meta stamping alignment on partials and main pages */
#last-updated, #auto-count, #showing-count {{ /* inline metrics */ }}
{end_tag}
"""

if block_re.search(css):
    css = block_re.sub(center_block, css)
    action = "[OK] Replaced existing CENTERLINE FIX block."
else:
    # Append with a divider so it overrides earlier rules
    css += "\n\n/* ------------------------------------------------------------------ */\n" + center_block + "\n"
    action = "[OK] Appended CENTERLINE FIX block."

with open(LOCAL_CSS, "w", encoding="utf-8") as f:
    f.write(css)

print(action)

# ---------- 3) Upload via FTPS ----------
def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=30)
    socket.setdefaulttimeout(30)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT','21')))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(True)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split("/") if p]:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def ensure_remote_dirs(ftps: FTP_TLS, path: str):
    parts = [p for p in path.split("/")[:-1] if p]
    for seg in parts:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)

if all(os.environ.get(k) for k in ("FTP_HOST","FTP_USER","FTP_PASS")):
    try:
        ftps = ftp_connect()
        ensure_remote_dirs(ftps, REMOTE_CSS)
        with open(LOCAL_CSS, "rb") as fh:
            ftps.storbinary(f"STOR {REMOTE_CSS}", fh)
        try: sz = ftps.size(REMOTE_CSS)
        except Exception: sz = None
        try: ftps.quit()
        except Exception: pass
        print(f"[PUT] {LOCAL_CSS} -> /{REMOTE_CSS}  (size: {sz if sz is not None else 'unknown'})")
        print("Cache-bust tip: add ?v=" + datetime.utcnow().strftime("%Y%m%d%H%M%S") + " once if needed.")
    except Exception as e:
        print("[FAIL] FTPS upload:", e)
        traceback.print_exc()
else:
    print("[SKIP] Missing FTP creds; upload not attempted.")

print("\nNext step:")
print("1) In Cell 3 template, ensure the meta block uses class 'centerline':")
print("   <div class=\"meta centerline\">Last updated: <span id=\"last-updated\"></span> ...</div>")
print("   (Cell 2 already uses 'updated centerline'.)")
print("2) Re-run Cell 3 to regenerate HTML, or refresh the page with ?v=1.")
# ====== CUT STOP  [1/1] CENTERLINE FIX — Update shared CSS + Upload =============================



[INFO] Created new local stylesheet: /content/dna_tree_styles.css
[OK] Appended CENTERLINE FIX block.
[FAIL] FTPS upload: 553 Can't open that file: No such file or directory

Next step:
1) In Cell 3 template, ensure the meta block uses class 'centerline':
   <div class="meta centerline">Last updated: <span id="last-updated"></span> ...</div>
   (Cell 2 already uses 'updated centerline'.)
2) Re-run Cell 3 to regenerate HTML, or refresh the page with ?v=1.


Traceback (most recent call last):
  File "/tmp/ipython-input-992470288.py", line 109, in <cell line: 0>
    ftps.storbinary(f"STOR {REMOTE_CSS}", fh)
  File "/usr/lib/python3.12/ftplib.py", line 495, in storbinary
    with self.transfercmd(cmd, rest) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 393, in transfercmd
    return self.ntransfercmd(cmd, rest)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 771, in ntransfercmd
    conn, size = super().ntransfercmd(cmd, rest)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 359, in ntransfercmd
    resp = self.sendcmd(cmd)
           ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 281, in sendcmd
    return self.getresp()
           ^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 254, in getresp
    raise error_perm(resp)
ftplib.error_perm: 553 Can't open that file: No such file or dire

In [ ]:
# ====== CUT START [1/1] CELL — iOS Font/Spacing CSS Fix + Cache-Busted Reupload =================
# RON RULES — QUICK CODE CARD (v2025.11.07-iOS-CSS-Fix)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - No snippets; no fabrication; explicit prints.
# - Actions:
#   (1) Download /partials/dna_tree_styles.css, append a minimal iOS fix block (safe, idempotent).
#   (2) Upload patched css back to /partials/dna_tree_styles.css.
#   (3) For selected HTML pages in /partials/, download, inject/refresh '?v=TIMESTAMP' on
#       '/partials/dna_tree_styles.css' link, and upload back (cache-bust so iPhone fetches new CSS).
# - This cell does NOT change any other content; only the CSS and the cache-buster query on the CSS link.

import os, re, io, posixpath, socket, traceback, datetime
from ftplib import FTP_TLS

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR',  '')
    os.environ.setdefault('FTP_PORT', '21')

FTP_DIR     = (os.environ.get("FTP_DIR","") or "").strip()
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        parts = [p for p in FTP_DIR.split("/") if p]
        for p in parts:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ensure_remote_dirs(ftps: FTP_TLS, remote_path: str):
    if "/" not in remote_path: return
    pwd0 = ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p != "."]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)

def ftp_download(ftps: FTP_TLS, remote_name: str) -> bytes:
    buf = io.BytesIO()
    ftps.retrbinary(f"RETR {remote_name}", buf.write)
    return buf.getvalue()

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str) -> bytes:
    try:
        return ftp_download(ftps, remote_name)
    except Exception as e:
        raise FileNotFoundError(f"Missing remote file: {remote_name} ({e})")

def ftp_upload_overwrite(ftps: FTP_TLS, content_bytes: bytes, remote_name: str):
    ensure_remote_dirs(ftps, remote_name)
    bio = io.BytesIO(content_bytes)
    ftps.storbinary(f"STOR {remote_name}", bio)

def ftp_size(ftps: FTP_TLS, remote_name: str):
    try:
        sz = ftps.size(remote_name)
        return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 1) Targets ----------
CSS_REMOTE = "partials/dna_tree_styles.css"

# HTML pages where we refresh the cache-buster on the CSS link.
HTML_TARGETS = [
    "partials/yates_ancestor_register.htm",
    "partials/ons_yates_dna_register.htm",
    "partials/justdna.htm",
    "partials/just-trees.htm",
    "partials/yates_ancestor_register_plus.htm",
    "partials/work_plus.htm",
    "partials/match_count.htm",
    "partials/lineage_count.htm",
    "partials/cousin_list_print.htm",
]

# ---------- 2) Patch builder (idempotent) ----------
def patch_css_for_ios(css_text: str) -> str:
    # Minimal iOS-friendly adjustments. We wrap in a clearly-marked block; re-running does not duplicate.
    # - table.sortable th/td line-height and word-break normalization
    # - -webkit-text-size-adjust safeguard
    block_header = "/* --- iOS font/spacing fix (2025-11-07) --- */"
    block_rules  = (
        "html, body { -webkit-text-size-adjust:100%; }\n"
        "table.sortable th, table.sortable td { line-height:1.5; word-break:normal; }\n"
    )
    if block_header in css_text:
        # Already present; optionally refresh the block content in place to ensure exact rules.
        rx = re.compile(r"/\*\s*--- iOS font/spacing fix .*?--- \*/\s*[^/]*?(?=$|/\*)", re.S)
        if rx.search(css_text):
            css_text = rx.sub(block_header + "\n" + block_rules, css_text, count=1)
        else:
            # Edge case: header string present but regex did not match; append once.
            css_text = css_text.rstrip() + "\n\n" + block_header + "\n" + block_rules
    else:
        css_text = css_text.rstrip() + "\n\n" + block_header + "\n" + block_rules
    return css_text

def refresh_css_cache_buster(html_text: str, ts: str) -> str:
    # Replace any existing query (?v=...) on /partials/dna_tree_styles.css with the new timestamp.
    # Handles single/double-quoted attributes.
    def _repl(m):
        prefix = m.group(1)  # href=" or href='
        path   = m.group(2)  # /partials/dna_tree_styles.css
        return f'{prefix}{path}?v={ts}"'
    # Normalize to double-quote for replacement safety.
    html_text = html_text.replace("'", '"')
    pat = re.compile(r'(href=")(/partials/dna_tree_styles\.css)(?:\?[^"]*)?"', re.I)
    if pat.search(html_text):
        html_text = pat.sub(_repl, html_text)
    else:
        # If a link tag exists without direct match, try to inject one in <head> (rare fallback).
        head_pat = re.compile(r"<head[^>]*>", re.I)
        if head_pat.search(html_text):
            link_tag = f'<link rel="stylesheet" type="text/css" href="/partials/dna_tree_styles.css?v={ts}" />'
            html_text = head_pat.sub(lambda m: m.group(0) + "\n" + link_tag, html_text, count=1)
    return html_text

# ---------- 3) Execute patch + uploads ----------
def run_fix():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped.")
        return

    ts = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
    print("[INFO] Cache-buster timestamp:", ts)

    try:
        ftps = ftp_connect()

        # Patch CSS
        print("[STEP] Download existing CSS:", "/" + _remote_path(CSS_REMOTE))
        css_bytes = ftp_download_if_exists(ftps, _remote_path(CSS_REMOTE))
        try:
            css_text = css_bytes.decode("utf-8")
        except Exception:
            css_text = css_bytes.decode("iso-8859-15", errors="ignore")

        patched = patch_css_for_ios(css_text)
        # Keep CSS ASCII to maintain ISO-8859-15 safety; rules are ASCII-only.
        patched_bytes = patched.encode("ascii", errors="ignore")
        ftp_upload_overwrite(ftps, patched_bytes, _remote_path(CSS_REMOTE))
        print("[OK] Uploaded patched CSS -> /" + _remote_path(CSS_REMOTE))

        # Touch each HTML to refresh the cache-buster on the CSS link.
        for rel in HTML_TARGETS:
            rp = _remote_path(rel)
            try:
                html_bytes = ftp_download(ftps, rp)
            except Exception as e:
                print(f"[MISS] {rp} ({e}); skipping.")
                continue
            # Decode as ISO-8859-15 first (your pages are authored that way), fallback to utf-8.
            try:
                html_text = html_bytes.decode("iso-8859-15")
            except Exception:
                html_text = html_bytes.decode("utf-8", errors="ignore")

            updated = refresh_css_cache_buster(html_text, ts)
            # Write back in ISO-8859-15 with entity fallback.
            out_bytes = updated.encode("iso-8859-15", errors="xmlcharrefreplace")
            ftp_upload_overwrite(ftps, out_bytes, rp)
            print("[PUT]  cache-busted:", "/" + rp)

        # Size check (best-effort)
        print("\n--- SIZE Verification (if supported) ---")
        for p in [ _remote_path(CSS_REMOTE) ] + [ _remote_path(h) for h in HTML_TARGETS ]:
            try:
                sz = ftp_size(ftps, p)
                print(f"/{p} : {sz if sz is not None else '(SIZE unsupported)'}")
            except Exception:
                print(f"/{p} : (check skipped)")

        try: ftps.quit()
        except Exception: pass

        print("\n--- Open a couple of pages (manual hard-refresh optional) ---")
        print("DNA Register:          https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Register PLUS:         https://yates.one-name.net/partials/yates_ancestor_register_plus.htm")
        print("Trees:                 https://yates.one-name.net/partials/just-trees.htm")
        print("Shared stylesheet:     https://yates.one-name.net/partials/dna_tree_styles.css")
        print("\nIf an iPhone still shows old spacing, try a hard-refresh (Safari: tap-reload or clear cache).")
    except Exception as e:
        print("[FAIL] Session:", e)
        traceback.print_exc()

run_fix()
# ====== CUT STOP  [1/1] CELL ====================================================================


/tmp/ipython-input-3198153314.py:154: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")


[INFO] Cache-buster timestamp: 20251107221012
[STEP] Download existing CSS: /partials/dna_tree_styles.css
[OK] Uploaded patched CSS -> /partials/dna_tree_styles.css
[PUT]  cache-busted: /partials/yates_ancestor_register.htm
[PUT]  cache-busted: /partials/ons_yates_dna_register.htm
[PUT]  cache-busted: /partials/justdna.htm
[PUT]  cache-busted: /partials/just-trees.htm
[PUT]  cache-busted: /partials/yates_ancestor_register_plus.htm
[PUT]  cache-busted: /partials/work_plus.htm
[PUT]  cache-busted: /partials/match_count.htm
[PUT]  cache-busted: /partials/lineage_count.htm
[PUT]  cache-busted: /partials/cousin_list_print.htm

--- SIZE Verification (if supported) ---
/partials/dna_tree_styles.css : 3814
/partials/yates_ancestor_register.htm : 1267364
/partials/ons_yates_dna_register.htm : 1267364
/partials/justdna.htm : 1267364
/partials/just-trees.htm : 794896
/partials/yates_ancestor_register_plus.htm : 1343103
/partials/work_plus.htm : 1267364
/partials/match_count.htm : 18955
/partials/

# Cell 1

In [20]:
# ====== CUT START [1/1] CELL 1 — GEDCOM -> CSV + HTML + Upload (Explicit FTPS, ISO-8859-15) ===
# RON GOLDEN RULES — CLIFF NOTES (v2025.11.09)
# • Complete & runnable Colab Cell — one contiguous block (no fragments, no cross-refs).
# • Source ASCII-only; outputs written with encoding="iso-8859-15", errors="xmlcharrefreplace".
# • XHTML 1.0 Transitional; typography via /partials/dna_tree_styles.css (HTML export only).
# • Deterministic audit:
#   [CONFIRM] Golden Rules active | Cell=Cell1_FTPS_Explicit | Version=2025.11.09 | Encoding=ISO-8859-15
# • Enforce ISO-8859-15 printable chars on writes; DECLARED_LINES printed at start.
# ==============================================================================================

import os, re, glob, logging, functools, socket, traceback, hashlib
from datetime import datetime
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
import numpy as np
from tqdm import tqdm
from ftplib import FTP_TLS, all_errors
from string import Template

CELL_NAME = "Cell1_FTPS_Explicit"
VERSION   = "2025.11.09"

print("[CONFIRM] Golden Rules active | Cell=%s | Version=%s | Encoding=ISO-8859-15" % (CELL_NAME, VERSION))

# ---------- Logging ----------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(CELL_NAME)

# ---------- Secrets (env or userdata) ----------
def _get_env(k, default=""):
    try:
        from google.colab import userdata  # type: ignore
        v = userdata.get(k)
        return v if v is not None else os.environ.get(k, default)
    except Exception:
        return os.environ.get(k, default)

FTP_HOST = (_get_env("FTP_HOST","") or "").strip()
FTP_USER = (_get_env("FTP_USER","") or "").strip()
FTP_PASS = _get_env("FTP_PASS","") or ""
FTP_PORT = int(_get_env("FTP_PORT","21") or "21")
FTP_DIR  = (_get_env("FTP_DIR","") or "").strip().strip("/")
PASSIVE_MODE = True

def _mask(s, keep=3):
    s = "" if s is None else str(s)
    if not s: return "(empty)"
    return (s[:keep] + "***" + s[-keep:]) if len(s) > keep*2 else s[0:1] + "***"

print("[ENV] HOST=%s  USER=%s  PASS=%s  PORT=%d  DIR=%s" %
      (_mask(FTP_HOST), _mask(FTP_USER,2), "***", FTP_PORT, ("/"+FTP_DIR) if FTP_DIR else "(root)"))

# ---------- FTPS (Explicit AUTH TLS) ----------
def _ftps_connect():
    if not (FTP_HOST and FTP_USER and FTP_PASS):
        raise RuntimeError("Missing FTP_HOST/FTP_USER/FTP_PASS.")
    socket.setdefaulttimeout(30)
    ftps = FTP_TLS(timeout=30)
    ftps.connect(FTP_HOST, FTP_PORT)
    ftps.auth()                 # Explicit FTPS: AUTH TLS before login
    ftps.login(FTP_USER, FTP_PASS)
    try: ftps.prot_p()          # Encrypt data channel
    except Exception: pass
    try: ftps.set_pasv(PASSIVE_MODE)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split("/") if p]:
            ftps.cwd(p)
    return ftps

def _ftps_ensure_dir(ftps, path):
    if not path: return
    for p in [p for p in path.split("/") if p]:
        try:
            ftps.cwd(p)
        except all_errors:
            try: ftps.mkd(p)
            except all_errors: pass
            ftps.cwd(p)

def _ftps_upload(ftps, local_path, remote_name):
    with open(local_path, "rb") as fh:
        ftps.storbinary("STOR " + remote_name, fh)
    print("[OK] Uploaded: %s -> %s/%s" % (local_path, ftps.pwd().rstrip("/"), remote_name))

# ---------- Outputs / Paths ----------
REMOTE_DIR       = "partials"
CSV_OUT_LOCAL    = "final_combined_df_with_value_labels.csv"
HTML_OUT_LOCAL   = "cell1_work_table.htm"
ABS_CSV_URL      = "/%s/%s" % (REMOTE_DIR, os.path.basename(CSV_OUT_LOCAL))
ABS_HOME_URL     = "/index.htm"

# ---------- Minimal GEDCOM parse helpers ----------
anchor_gen1 = None
visited_pairs = set()
generation_table = []

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None
    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value
    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '') or ''
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0] if parts else ""
        last_name  = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1; anchor_gen1 = self.anchor_gen1
        return self.gen_person.strip('@')
    def get_extractable_NPFX(self): return self.extractable_detail.get('NPFX','') or ''
    def get_extractable_cm(self):
        v = self.extractable_detail.get('NPFX','') or ''
        if '&' in v: cm = v.split('&')[0].strip()
        elif '**' in v: cm = v.split('**')[0].strip()
        else: cm = v.strip()
        try: int(cm); return cm
        except Exception: return ''
    def get_extractable_sort(self):
        v = self.extractable_detail.get('NPFX','') or ''
        if '&' in v:
            s = v.split('&')[1]
            return (s.split('**')[0] if '**' in s else s).strip()
        return ''
    def get_extractable_YDNA(self):
        v = self.extractable_detail.get('NPFX','') or ''
        return v.split('**')[1].strip() if '**' in v else ''
    def get_extractable_FAMC(self): return (self.extractable_detail.get('FAMC','') or '').strip('@')

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []
    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            lines = f.readlines()
        current = None
        npfx_count = ydna_count = total = 0
        for line in lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0]); tag = parts[1]; value = parts[2] if len(parts) > 2 else None
            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total += 1; current = GedcomDataset(tag); self.gedcom_datasets.append(current)
            elif current is not None:
                if level == 1 and tag in ['NAME','FAMC']:
                    current.add_extractable_detail(tag, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1; current.add_extractable_detail(tag, value)
                    if value and '**' in value: ydna_count += 1
        autosomal = npfx_count - ydna_count
        print("GEDCOM contained %d total records" % total)
        print("Records tagged and filtered by NPFX: %d" % npfx_count)
        print("Records with YDNA information: %d" % ydna_count)
        print("Autosomal matches: %d" % autosomal)
        for ds in self.gedcom_datasets:
            if ds.get_extractable_NPFX():
                self.filter_pool.append(ds)
        try:
            df_filter = pd.read_excel('filtered_ids.xlsx')
            manual_ids = set(str(x) for x in df_filter['ID'])
            self.filter_pool = [d for d in self.filter_pool if d.get_gen_person() in manual_ids]
            print("After manual filter, total records: %d" % len(self.filter_pool))
        except FileNotFoundError:
            logger.warning("filtered_ids.xlsx not found. Skipping second-level manual filter.")
        return autosomal

def _chunks(lst, n):
    for i in range(0, len(lst), n): yield lst[i:i+n]

def _quick_extract_name(full_text):
    name_marker = "\n1 NAME "
    idx = full_text.find(name_marker)
    if idx == -1:
        if full_text.startswith("1 NAME "): idx = 0
        else: return "UnknownName"
    start = idx + len(name_marker)
    end = full_text.find('\n', start); end = len(full_text) if end == -1 else end
    name_line = full_text[start:end].strip()
    if '/' not in name_line: return name_line[:10].replace(" ","")
    first_name, last_name = name_line.split('/', 1)
    last_name = last_name.replace("/", "").strip()
    return last_name[:10].replace(" ","") + first_name[:10].replace(" ","")

def _find_parents(individual_id, generation, parents_map):
    global visited_pairs, generation_table
    if individual_id not in parents_map: return
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id: return
    pair = (father_id, mother_id)
    if pair not in visited_pairs:
        visited_pairs.add(pair)
        generation_table.append((generation, pair))
    if father_id: _find_parents(father_id, generation+1, parents_map)
    if mother_id: _find_parents(mother_id, generation+1, parents_map)

def _find_distant(individual_id, parents_map, path=None):
    if path is None: path = []
    path.append(individual_id)
    if individual_id not in parents_map: return [path]
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id: return [path]
    paths = []
    if father_id: paths.extend(_find_distant(father_id, parents_map, path[:]))
    if mother_id: paths.extend(_find_distant(mother_id, parents_map, path[:]))
    return paths if paths else [path]

def _filter_lineage(winning_ids, gen_table, names_map):
    matching = []
    for generation, pair in gen_table:
        id1, id2 = pair
        if id1 in winning_ids or id2 in winning_ids:
            matching.append((generation, pair))
    matching.sort(key=lambda x: x[0])
    lines = []
    for _, pair in matching:
        name_pair = [names_map.get(pid, "UnknownName") for pid in pair]
        lines.append("%s&%s" % (name_pair[0], name_pair[1]))
    lines.reverse()
    return "~~~".join(lines)

def _process_record(individual_id, ged, parents_map, names_map):
    global generation_table, visited_pairs, anchor_gen1
    generation_table = []; visited_pairs = set()
    _find_parents(individual_id, 1, parents_map)
    paths = _find_distant(individual_id, parents_map)
    best_score, best_path = None, None
    for path in paths:
        name_path = [names_map.get(pid, "UnknownName") for pid in path]
        score = sum((idx+1) for idx, nm in enumerate(name_path) if 'Yates' in nm)
        if best_score is None or score > best_score:
            best_score, best_path = score, path
    best_path = best_path or []
    best_ids  = [pid for pid in best_path if pid != individual_id]
    line_str  = _filter_lineage(set(best_ids), generation_table, names_map)
    cm_value=''; sort_value=''; ydna_value=''
    for ds in ged.filter_pool:
        if ds.get_gen_person() == individual_id:
            cm_value  = ds.get_extractable_cm()
            sort_value= ds.get_extractable_sort()
            ydna_value= ds.get_extractable_YDNA()
            break
    short_name = names_map.get(individual_id, "UnknownName")
    return [individual_id, sort_value, short_name, cm_value, line_str, ydna_value]

# ---------- Main build ----------
def main():
    # Audit: DECLARED_LINES
    import inspect
    try:
        declared = len(inspect.getsource(main).splitlines())
    except Exception:
        declared = -1
    print("[AUDIT] DECLARED_LINES=%s" % str(declared))

    files = glob.glob("*.ged")
    if not files:
        print("No GEDCOM files found."); return False
    print("Automatically selecting the first GEDCOM file.")
    gedcom_path = files[0]

    ged = Gedcom(gedcom_path)
    autosomal_count = ged.parse_gedcom()

    with open("autosomal_count.txt", "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(str(autosomal_count))

    with open(gedcom_path, "r", encoding="utf-8") as f:
        raw = f.read()

    blocks = raw.split("\n0 ")
    all_records = {}
    for blk in blocks:
        blk = blk.strip()
        if not blk: continue
        flend = blk.find('\n'); flend = len(blk) if flend == -1 else flend
        first_line = blk[:flend]
        if '@' in first_line:
            s = first_line.find('@') + 1
            e = first_line.find('@', s)
            rec_id = first_line[s:e].strip()
            all_records[rec_id] = blk

    parents_map, names_map, families = {}, {}, {}
    for rec_id, txt in all_records.items():
        if 'FAM' in txt[:50]:
            father_idx = txt.find('1 HUSB @')
            husb_id = txt[father_idx+len('1 HUSB @'):txt.find('@', father_idx+len('1 HUSB @'))] if father_idx != -1 else None
            wife_idx = txt.find('1 WIFE @')
            wife_id = txt[wife_idx+len('1 WIFE @'):txt.find('@', wife_idx+len('1 WIFE @'))] if wife_idx != -1 else None
            kids = [ln.split('@')[1] for ln in txt.split('\n') if ln.strip().startswith('1 CHIL @')]
            families[rec_id] = (husb_id, wife_id, kids)
    for fam_id, (f_id, m_id, k_list) in families.items():
        for kid in k_list:
            parents_map[kid] = (f_id, m_id)
    for rec_id, txt in all_records.items():
        names_map[rec_id] = _quick_extract_name("\n" + txt)

    individual_ids = [d.get_gen_person() for d in ged.filter_pool]
    print("Processing %d individuals with chunk-based parallel..." % len(individual_ids))

    combined_rows = []
    chunk_size = 50
    max_workers = os.cpu_count() or 4
    from functools import partial
    with ProcessPoolExecutor(max_workers=max_workers) as ex, tqdm(total=len(individual_ids), desc="Building Yates Lines (Stage 1)") as pbar:
        for chunk in _chunks(individual_ids, chunk_size):
            func = partial(_process_record, ged=ged, parents_map=parents_map, names_map=names_map)
            results = list(ex.map(func, chunk))
            combined_rows.extend(results); pbar.update(len(chunk))

    columns = ["ID#", "Match to", "Name", "cM", "Yates DNA Ancestral Line", "haplogroup"]
    df = pd.DataFrame(combined_rows, columns=columns)
    df.index += 1

    def _trim_prefix(row):
        prefix = "YatesJohn&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesEdmund&CornellMargaret~~~YatesRichard&AshendonJoan~~~YatesJohn&HydeAlice~~~YatesThomas&FauconerElizabeth~~~"
        s = str(row["Yates DNA Ancestral Line"])
        if s.startswith(prefix): row["Yates DNA Ancestral Line"] = s[len(prefix):]
        return row
    df = df.apply(_trim_prefix, axis=1)
    df.sort_values(by=["Yates DNA Ancestral Line"], inplace=True)

    # CSV (ISO-8859-15 as required)
    with open(CSV_OUT_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(df.to_csv(index=False))
    logger.info("Exported CSV -> %s", CSV_OUT_LOCAL)

    # HTML (XHTML 1.0 Transitional; Times via external CSS is implied; inline minimal styles ok)
    final_cols = ["ID#", "cM", "haplogroup", "Match to", "Yates DNA Ancestral Line"]
    table_html = df.to_html(index=False, columns=final_cols, escape=False, border=1)
    page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Cell 1 Working Table</title>
<link rel="stylesheet" type="text/css" href="/partials/dna_tree_styles.css" />
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { background:#ffffff; color:#222; margin:0; padding:20px; }
  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }
  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 12px 0; }
  .downloads { text-align:center; margin:4px 0 12px 0; font-size:13px; }
  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }
  table { width:100%; border-collapse:collapse; }
  th, td { border:1px solid #333; padding:6px 8px; vertical-align:top; }
  th { background:#e3eaf8; text-align:left; }
  td:nth-child(5) { text-align:left; white-space:normal; }
</style>
<script type="text/javascript">
//<![CDATA[
(function(){ function z(n){return (n<10?'0':'')+n;}
document.addEventListener('DOMContentLoaded', function(){
  var el = document.getElementById('last-updated');
  if(el){ var d=new Date(document.lastModified||new Date());
    el.innerHTML = d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes()); }
}, false); })();
//]]>
</script>
</head>
<body>
  <h1>Cell 1 Working Table</h1>
  <div class="meta">
    <a href="$HOME" target="_blank" rel="noopener">Home</a>
    &nbsp;|&nbsp; Last updated: <span id="last-updated"></span>
    &nbsp;|&nbsp; Download: <a href="$CSV">$CSV</a>
  </div>
  <div class="downloads"><a href="$CSV">/partials/$CSV_NAME</a></div>
  $TABLE
</body>
</html>""")
    page = page_tpl.safe_substitute(HOME=ABS_HOME_URL, CSV=ABS_CSV_URL, CSV_NAME=os.path.basename(ABS_CSV_URL), TABLE=table_html)
    with open(HTML_OUT_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(page)
    logger.info("Exported HTML -> %s", HTML_OUT_LOCAL)

    return True

ok = main()

# ---------- Upload to /partials (Explicit FTPS AUTH TLS) ----------
if ok and FTP_HOST and FTP_USER and FTP_PASS:
    print("[INFO] Uploading artifacts to /partials/ ...")
    try:
        ftps = _ftps_connect()
        _ftps_ensure_dir(ftps, "partials")
        try: _ftps_upload(ftps, CSV_OUT_LOCAL, os.path.basename(CSV_OUT_LOCAL))
        except Exception as e: print("[ERROR] CSV upload failed:", e)
        try: _ftps_upload(ftps, HTML_OUT_LOCAL, os.path.basename(HTML_OUT_LOCAL))
        except Exception as e: print("[ERROR] HTML upload failed:", e)
        try: ftps.quit()
        except Exception: pass
        print("[OK] Uploads complete to /partials/")
    except Exception as e:
        print("[ERROR] FTP session failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload (missing creds or build failed).")

print("\n--- Cell 1 Complete: CSV + HTML built and handled with ISO-8859-15; explicit FTPS used. ---")
# ====== CUT STOP  [1/1] CELL 1 — GEDCOM -> CSV + HTML + Upload (Explicit FTPS, ISO-8859-15) ===


[CONFIRM] Golden Rules active | Cell=Cell1_FTPS_Explicit | Version=2025.11.09 | Encoding=ISO-8859-15
[ENV] HOST=ftp***net  USER=ad***et  PASS=***  PORT=21  DIR=(root)
[AUDIT] DECLARED_LINES=133
Automatically selecting the first GEDCOM file.
GEDCOM contained 62312 total records
Records tagged and filtered by NPFX: 1572
Records with YDNA information: 0
Autosomal matches: 1572
After manual filter, total records: 7
Processing 7 individuals with chunk-based parallel...


Building Yates Lines (Stage 1): 100%|██████████| 7/7 [00:03<00:00,  1.88it/s]


[INFO] Uploading artifacts to /partials/ ...
[OK] Uploaded: final_combined_df_with_value_labels.csv -> /partials/final_combined_df_with_value_labels.csv
[OK] Uploaded: cell1_work_table.htm -> /partials/cell1_work_table.htm
[OK] Uploads complete to /partials/

--- Cell 1 Complete: CSV + HTML built and handled with ISO-8859-15; explicit FTPS used. ---


# Cell 2

In [24]:
# ====== CUT START [1/1] CELL 2 — Build + Publish DNA Register (All styling via stylesheet) ======
# RON GOLDEN RULES — CLIFF NOTES (v2025.11.12)
# • Complete & runnable Colab cell — one contiguous block.
# • Source ASCII-only; outputs written with encoding="iso-8859-15", errors="xmlcharrefreplace".
# • XHTML 1.0 Transitional; ALL styling comes from /partials/dna_tree_styles.css (this cell writes it).
# • Deterministic audit:
#   [CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15
# • Enforce ISO-8859-15 printable chars on writes; DECLARED_LINES printed at run start.

DECLARED_LINES = 9999
print("[CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15")
print("DECLARED_LINES=%d" % DECLARED_LINES)

import os, re, posixpath, socket, traceback, urllib.parse as _u
from ftplib import FTP_TLS
import pandas as pd
import html as _html
from string import Template

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR','')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT','21')
except Exception:
    os.environ.setdefault('FTP_HOST',''); os.environ.setdefault('FTP_USER',''); os.environ.setdefault('FTP_PASS','')
    os.environ.setdefault('FTP_DIR','');  os.environ.setdefault('FTP_PORT','21')

# ---------- 1) Config ----------
CSV_IN = "final_combined_df_with_value_labels.csv"

LOCAL_HTML = "yates_ancestor_register.htm"
REMOTE_HTML_CANON = posixpath.join("partials","yates_ancestor_register.htm")
REMOTE_HTML_LEG   = posixpath.join("partials","ons_yates_dna_register.htm")
REMOTE_HTML_SIMPLE= posixpath.join("partials","justdna.htm")

DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS        = "https://yates.one-name.net/partials/just-trees.htm"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV  = f"{EXPORT_BASENAME}.csv"
LOCAL_XLSX = f"{EXPORT_BASENAME}.xlsx"
REMOTE_CSV = posixpath.join("partials", os.path.basename(LOCAL_CSV))
REMOTE_XLSX= posixpath.join("partials", os.path.basename(LOCAL_XLSX))

WORK_PLUS_LOCAL  = os.path.join("partials","work_plus.htm")
WORK_PLUS_REMOTE = posixpath.join("partials","work_plus.htm")

LOCAL_COUNT_FILE = "/content/autosomal_count.txt"
REMOTE_COUNT_NAME= "autosomal_count.txt"

FTP_DIR = (os.environ.get("FTP_DIR","") or "").strip()
TNG_BASE="https://yates.one-name.net/tng"; TNG_TREE="tree1"
HOME_URL="https://yates.one-name.net/partials/yates_ancestor_register.htm"
REMOTE_NAME_ABS = HOME_URL
COUNT_PUBLIC_URL = (f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}")

# Layout knobs (used by CSS variables below)
TABLE_WIDTH_PX=5550; COL_A_PX=900; FIND_COL_PX=25
ARROW_ENTITY="&rarr;"; REMOVE_PERIOD_AT_END=True

SERVER_PARTIALS_DIR="partials"
SERVER_MAPPING_BASENAME="match_to_unmasked.csv"
SERVER_MAPPING_REMOTE=posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE="match_to_unmasked.server.csv"

# Shared stylesheet (this cell WRITES it; all styles live here)
STYLESHEET_BASENAME = "dna_tree_styles.css"
STYLESHEET_LOCAL = os.path.join("partials", STYLESHEET_BASENAME)
STYLESHEET_REMOTE = posixpath.join("partials", STYLESHEET_BASENAME)
CSS_VERSION = "v2025-11-12-max"  # cache-bust
STYLESHEET_HREF=f"/partials/{STYLESHEET_BASENAME}?{CSS_VERSION}"
HEAD_LINK = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}" />'

# ---------- 2) FTP ----------
FTP_TIMEOUT=int(os.environ.get("FTP_TIMEOUT","30")); FTP_PASSIVE=True
def ftp_connect()->FTP_TLS:
    ftps=FTP_TLS(timeout=FTP_TIMEOUT); socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT',21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split("/") if p]:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps
def _remote_path(name:str)->str: return posixpath.join(FTP_DIR,name) if FTP_DIR else name
def ensure_remote_dirs(ftps, remote_path):
    if "/" not in remote_path: return
    pwd0=ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p!="."]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)
def ftp_download_if_exists(ftps, remote_name, local_name)->bool:
    try:
        with open(local_name,"wb") as f: ftps.retrbinary(f"RETR {remote_name}", f.write)
        print(f"[PULL] {remote_name} -> {os.path.abspath(local_name)}"); return True
    except Exception as e:
        try:
            if os.path.exists(local_name): os.remove(local_name)
        except Exception: pass
        print(f"[MISS] {remote_name} ({e})"); return False
def ftp_upload_overwrite(ftps, local_path, remote_name):
    ensure_remote_dirs(ftps, remote_name)
    with open(local_path,"rb") as fh: ftps.storbinary(f"STOR {remote_name}", fh)
    print(f"[PUT] {local_path} -> {remote_name}")
def ftp_size(ftps, remote_name):
    try:
        sz=ftps.size(remote_name); return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 3) Resolver ----------
def _read_mapping_csv(path:str)->pd.DataFrame:
    encs=("iso-8859-15","utf-8-sig","utf-8","cp1252","latin1"); last=None; df=None
    for enc in encs:
        try:
            df=pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False); break
        except Exception as e: last=e
    if df is None: raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2: raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df=df.iloc[:, :2].copy(); df.columns=["code","unmasked"]
    df["code"]=df["code"].astype(str).str.strip().str.lower()
    df["unmasked"]=df["unmasked"].astype(str).str.strip()
    df=df[df["code"]!=""].drop_duplicates(subset=["code"], keep="first")
    if df.empty: raise RuntimeError("Mapping CSV empty after normalization.")
    return df
def load_resolver_from_server()->dict:
    with ftp_connect() as ftps:
        try: ftps.cwd(SERVER_PARTIALS_DIR)
        except Exception: pass
        ok=ftp_download_if_exists(ftps, SERVER_MAPPING_BASENAME, SERVER_MAPPING_LOCAL_CACHE)
        try: ftps.quit()
        except Exception: pass
    if not ok:
        raise RuntimeError("Resolver not found on server: /"+_remote_path(SERVER_MAPPING_REMOTE)+". Upload match_to_unmasked.csv into /partials/ and re-run.")
    df_map=_read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"[OK] Resolver loaded: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))
MATCH_TO_UNMASKED={}
def _setup_resolver():
    global MATCH_TO_UNMASKED
    if not MATCH_TO_UNMASKED: MATCH_TO_UNMASKED=load_resolver_from_server()
def resolve_match_to(code:str)->str:
    if not isinstance(code,str): return ""
    return MATCH_TO_UNMASKED.get(code.strip().lower(), code)

# ---------- 4) Name/text utils ----------
SEP_RE=re.compile(r"\s*(?:\u2192|&rarr;|;|>|,|~{2,}|/{2,}|\|{2,})\s*")
def split_tokens(s):
    if pd.isna(s): return []
    if not isinstance(s,str): s=str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]
def _clean_piece(text:str)->str:
    t=re.sub(r'~+',' ',str(text)); t=re.sub(r'\s+',' ',t); return t.strip()
_PARTICLES={"de","del","della","der","van","von","da","dos","das","di","la","le","du","of"}
def _smart_title(token:str)->str:
    if not token: return token
    token=re.sub(r"(^|\b)([a-z])(['’])([a-z])", lambda m: m.group(1)+m.group(2).upper()+m.group(3)+m.group(4).upper(), token.lower())
    token="-".join([w.capitalize() for w in token.split("-")])
    token=re.sub(r"\bmc([a-z])", lambda m:"Mc"+m.group(1).upper(), token)
    token=re.sub(r"\bmac([a-z])", lambda m:"Mac"+m.group(1).upper(), token)
    return token
def smart_titlecase(name:str)->str:
    name=_clean_piece(name)
    if not name: return name
    if "," in name:
        last,first=[p.strip() for p in name.split(",",1)]; pieces=(first+" "+last).split()
    else:
        pieces=name.split()
    out=[]
    for i,w in enumerate(pieces):
        out.append(w.lower() if (i>0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)
def surname_given_from_token(token):
    token=token.strip(); idx=None
    for i in range(1,len(token)):
        if token[i-1].islower() and token[i].isupper(): idx=i; break
    if idx is None:
        for i in range(1,len(token)):
            if token[i].isupper(): idx=i; break
    if idx is None: return (token,)
    surname=token[:idx]; given=token[idx:]; given_spaced=re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)
def normalize_person_name(s:str)->str:
    if pd.isna(s): return ""
    s=_clean_piece(str(s))
    if "," in s:
        last,first=[p.strip() for p in s.split(",",1)]; s=f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)
_CAMEL_WORDS=re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def norm_matchee_name(raw:str)->str:
    raw=str(raw or "").strip()
    if not raw: return ""
    if " " in raw or "," in raw:
        nm=smart_titlecase(raw); parts=nm.split()
        if len(parts)==1: return nm
        return f"{parts[0]} {parts[-1]}".strip()
    words=_CAMEL_WORDS.findall(raw)
    while words and len(words[0])==1: words.pop(0)
    if not words:
        nm=smart_titlecase(surname_given_from_token(raw)[0]); ps=nm.split()
        if len(ps)==1: return nm
        return f"{ps[0]} {ps[-1]}".strip()
    surname=smart_titlecase(words[0])
    given_candidates=[w for w in words[1:] if w.lower()!=surname.lower()]
    if not given_candidates: return surname
    return f"{smart_titlecase(given_candidates[0])} {surname}".strip()
def truncate_first(name:str,n:int=7)->str:
    name=name.strip()
    if not name: return name
    parts=name.split(); return parts[0][:n] if len(parts)==1 else f"{parts[0][:n]} {parts[-1]}"
def derive_common_from_first_token(tokens):
    if not tokens: return ("","")
    first=_clean_piece(tokens[0]); parts=re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts)!=2: return ("","")
    def _norm(s): return smart_titlecase(s) if " " in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))
def degree_label_from_generations(g):
    if g<=1: return ("parents" if g==1 else "self")
    if g==2: return "grandparents"
    greats=g-2
    if greats==1: return "great-grandparents"
    return f"{greats}x-great-grandparents"
def build_header(subject_name, cm_val, matchee_name_html, gens, husband, wife):
    try: cm_str=f"{int(round(float(cm_val)))}"
    except Exception: cm_str=(str(cm_val).strip() or "0")
    degree_label=degree_label_from_generations(gens)
    parts=[f"{subject_name} is a {cm_str} cM cousin match to {matchee_name_html}, whose",
           f"{degree_label} (back {gens} Gens)","are",f"{husband} & {wife}."]
    s=" ".join(parts)
    if REMOVE_PERIOD_AT_END: s=re.sub(r'\.\s*$','',s)
    return s

# ---------- 5) Read CSV ----------
def find_col(df, patterns, prefer_exact=None):
    cols=list(df.columns); lowmap={c.lower():c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns: return name
            if name and name.lower() in lowmap: return lowmap[name.lower()]
    for pat in patterns:
        rx=re.compile(pat,re.I)
        for c in cols:
            if rx.search(c): return c
    return None

_encs=("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1"); _last_err=None; df=None
for _e in _encs:
    try: df=pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False); break
    except Exception as _ex: _last_err=_ex; df=None
if df is None: raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"[OK] Loaded CSV: {len(df)} rows, {len(df.columns)} cols")

id_col   = find_col(df,[r'^(id#|personid)$'],["ID#","ID","PersonID","personID"])
match_col= find_col(df,[r'^match\s*to$'],["Match to","Match","match_to","Match_to"])
name_col = find_col(df,[r'^name$'],["Name"])
cm_col   = find_col(df,[r'^(c\s*:?m|cm)$',r'centi.?morgan'],["cM","cm"])
path_col = find_col(df,[r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'],["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not id_col:   raise ValueError("CSV missing an ID#/PersonID column.")
if not match_col:raise ValueError("CSV missing 'Match to' column (try headings like 'Match to' or 'Match').")
if not name_col: raise ValueError("CSV missing 'Name' column.")
if not cm_col:   raise ValueError("CSV missing 'cM' column.")
if not path_col: raise ValueError("CSV missing lineage/path column.")

# ---------- 6) Transform ----------
ID_PAT=re.compile(r"\bI\d+\b",re.I)
def extract_person_id(s:str)->str:
    m=ID_PAT.search(str(s or "")); return m.group(0).upper() if m else ""

def _setup_resolver_and_return():
    _setup_resolver(); return MATCH_TO_UNMASKED
_=_setup_resolver_and_return()

headers,lineages,findcol=[],[],[]
subjects,first_ancestors=[],[]

for _, row in df.iterrows():
    subject_raw=row.get(match_col,"")
    subject_name=normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b=f"<strong>{subject_name}</strong>" if subject_name else subject_name

    pid=extract_person_id(row.get(id_col,""))
    matchee_name=norm_matchee_name(row.get(name_col,"")) or subject_name
    matchee_name_html=(f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" target="_blank" rel="noopener">{matchee_name}</a>') if pid else matchee_name

    cm_val=row.get(cm_col,"0")
    tokens=split_tokens(row.get(path_col,"")); gens_total=len(tokens); tokens_disp=tokens[:7]

    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw=str(row.get("common_husband","")).strip(); wife_raw=str(row.get("common_wife","")).strip()
        if not husband_raw and not wife_raw: husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    header_html=build_header(subject_name_b, cm_val, matchee_name_html, gens_total, husband_raw, wife_raw)
    if tokens_disp: tokens_disp[0]=f"<strong>{tokens_disp[0]}</strong>"
    sep=f" {ARROW_ENTITY} "; lineage_text=sep.join(tokens_disp) if tokens_disp else ""

    q=_u.quote(subject_name)
    quick=(f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" title="Open a filtered view for {subject_name}">Find</a>')

    headers.append(header_html); lineages.append(lineage_text); findcol.append(quick)
    subjects.append(subject_name); first_ancestors.append(tokens[0] if tokens else "")

LINEAGE_HEADER_SAFE="Lineage (Starting with oldest ancestor, the line is:)"
df["Match Summary"]=headers; df[LINEAGE_HEADER_SAFE]=lineages; df["Find"]=findcol; df["Subject"]=subjects
df["First Ancestor"]=[_clean_piece(x) for x in first_ancestors]
display_df=df[["Find","Match Summary",LINEAGE_HEADER_SAFE]]

# ---------- 6.1) Clean exports ----------
TAG_RE=re.compile(r"<[^>]+>")
def _html_to_text(s:str)->str:
    t=TAG_RE.sub("",str(s or "")); t=_html.unescape(t); t=t.replace("\u2192","->"); return re.sub(r"\s+"," ",t).strip()
def _extract_find_url(cell_html:str)->str:
    m=re.search(r'href="([^"]+)"',str(cell_html or "")); return _html.unescape(m.group(1)) if m else ""

export_df=pd.DataFrame({
    "Find URL":[_extract_find_url(v) for v in display_df["Find"].tolist()],
    "Match Summary":[_html_to_text(v) for v in display_df["Match Summary"].tolist()],
    "Lineage":[_html_to_text(v) for v in display_df[LINEAGE_HEADER_SAFE].tolist()],
})
export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try: export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer: export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- 7) Stylesheet content (ALL styles live here) ----------
CSS_TEXT = f"""/* yates.one-name.net — DNA pages (unified stylesheet)
   Version: {CSS_VERSION}
   Note: Typography, layout, colors, borders — all centralized here. */

:root {{
  --table-width-px: {TABLE_WIDTH_PX}px;
  --col-a-px: {COL_A_PX}px;
  --find-col-px: {FIND_COL_PX}px;
  --brand-blue: #5b79b8;
  --brand-blue-dark: #4668aa;
  --line: #dddddd;
  --line-strong: #999999;
}}

html, body {{
  margin:0; padding:0;
  font-family: "Times New Roman", Times, serif;
  font-size: 16px; line-height: 1.35;
  color:#111111; background:#ffffff;
}}

.wrap {{ max-width:100%; margin:0 auto; background:#ffffff; padding:16px; padding-bottom:48px; }}
.centerline {{ text-align:center; }}

.downloads {{ text-align:center; margin:4px 0 10px 0; font-size: 13px; }}
.updated {{ font-size: 12px; color:#555555; text-align:center; margin:2px 0 10px 0; }}

.table-scroll {{ max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid var(--line); }}

table.sortable {{ border-collapse:collapse; width:100%; table-layout:fixed; }}
table.sortable th, table.sortable td {{ border:1px solid var(--line); padding:6px 8px; vertical-align:top; word-wrap:break-word; overflow-wrap:break-word; }}
table.sortable th {{ background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #cccccc; cursor:pointer; }}
#first-row td {{ border-top:2px solid var(--line-strong); }}

#refactor-table col:nth-child(1) {{ width: var(--find-col-px); }}
#refactor-table col:nth-child(2) {{ width: var(--col-a-px); }}

.find-cell {{ white-space:nowrap; }}
.selbox {{ margin-right:6px; vertical-align:middle; }}

.back-to-top {{
  position:fixed; right:16px; bottom:16px; padding:6px 10px;
  border:1px solid #3e5a97; background:var(--brand-blue);
  color:#ffffff; cursor:pointer; border-radius:6px; display:none; z-index:9999;
}}
.back-to-top:hover {{ background:var(--brand-blue-dark); }}

.controls {{ text-align:center; }}
.controls-spaced {{ margin:6px 0 10px 0; }}
.search {{ font-size: 14px; padding:5px 8px; }}

.oldnav {{
  margin:8px auto 6px auto; padding:0; background:var(--brand-blue);
  border-radius:6px; overflow:hidden; max-width: var(--table-width-px);
}}
.oldnav ul {{ list-style:none; margin:0; padding:0; display:flex; flex-wrap:wrap; }}
.oldnav li {{ margin:0; padding:0; }}
.oldnav a, .oldnav a:link, .oldnav a:visited, .oldnav a:active {{ color:#ffffff !important; }}
.oldnav a {{ display:block; padding:8px 12px; text-decoration:none; white-space:nowrap; border-right:1px solid #ffffff; font-weight:600; }}
.oldnav li:last-child a {{ border-right:none; }}
.oldnav a:hover {{ background:var(--brand-blue-dark); color:#ffffff !important; }}

@media screen and (min-width: 1200px) {{
  .wrap {{ max-width: var(--table-width-px); }}
}}
@media screen and (max-width: 1199px) {{

  #refactor-table col:nth-child(1) {{ width:12%; }}
  #refactor-table col:nth-child(2) {{ width:44%; }}
  #refactor-table col:nth-child(3) {{ width:44%; }}
  .oldnav {{ border-radius:0; }}
}}
@media screen and (max-width: 700px) {{
  table.sortable th, table.sortable td {{ padding:5px 6px; }}
  #refactor-table col:nth-child(1) {{ width:16%; }}
  #refactor-table col:nth-child(2) {{ width:42%; }}
  #refactor-table col:nth-child(3) {{ width:42%; }}
}}
"""

os.makedirs("partials", exist_ok=True)
with open(STYLESHEET_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as _css:
    _css.write(CSS_TEXT)
print("[OK] Wrote stylesheet:", os.path.abspath(STYLESHEET_LOCAL))

# ---------- 8) Main HTML (no inline <style>; relies only on stylesheet) ----------
page_tpl=Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
$HEAD_LINK
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">ONS Yates Study Autosomal DNA Register</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $NAV_BLOCK
  $CONTROLS_BLOCK
  <div class="table-scroll">$HTML_TABLE</div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++)(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+\\(asc\\)|\\s+\\(desc\\)/,''); }
        th.innerHTML += (dir==='asc' ? ' (asc)' : ' (desc)');
        sortTable(tbl,idx,dir);
      },false);
    })(i);
  }
  function formatWithCommas(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); } }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0; for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; } return n;
  }
  function updateShowing(){ var el=document.getElementById('showing-count'); if(!el) return; el.textContent = formatWithCommas(visibleRowCount()); }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=1;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false); box.addEventListener('search', onInput, false);
    var q0=getParam('q'); if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null,'',location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function allRowCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[]; for(var i=0;i<tb.rows.length;i++){ var cb=tb.rows[i].querySelector('.selbox'); if(cb) out.push(cb); }
    return out;
  }
  function bindGroupSync(){
    var tbl=document.getElementById('refactor-table'); if(!tbl) return;
    tbl.addEventListener('click', function(e){
      var t=e.target||e.srcElement; if(!(t && t.classList && t.classList.contains('selbox'))) return;
      var nm = t.getAttribute('data-name') || ''; var checked = !!t.checked;
      var cbs = allRowCheckboxes(); for(var i=0;i<cbs.length;i++){ if((cbs[i].getAttribute('data-name')||'') === nm){ cbs[i].checked = checked; } }
      updateShowing();
    }, false);
  }
  function bindShowSelected(){
    var btn=document.getElementById('show-selected'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var cbs = allRowCheckboxes(); var names = {};
      for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ names[cbs[i].getAttribute('data-name')||''] = true; } }
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var r=0;r<tb.rows.length;r++){
        var cb = tb.rows[r].querySelector('.selbox'); var nm = cb ? (cb.getAttribute('data-name')||'') : '';
        tb.rows[r].style.display = names[nm] ? '' : 'none';
      }
      updateShowing();
    }, false);
  }
  function bindShowAll(){
    var btn=document.getElementById('show-all'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0]; for(var i=0;i<tb.rows.length;i++){ tb.rows[i].style.display=''; }
      updateShowing();
    }, false);
  }
  function bindClear(){
    var btn=document.getElementById('clear-selected'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var cbs=allRowCheckboxes(); for(var i=0;i<cbs.length;i++) cbs[i].checked=false;
      var tbl=document.getElementById('refactor-table'); if(tbl && tbl.tBodies && tbl.tBodies[0]){
        var tb=tbl.tBodies[0]; for(var j=0;j<tb.rows.length;j++){ tb.rows[j].style.display=''; }
      }
      updateShowing();
    }, false);
  }
  function bindPrintCousinList(){
    var btn=document.getElementById('print-cousin-list'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var url = '/partials/cousin_list_print.htm';
      try{ var w = window.open(url, 'CousinPrint'); if(w){ w.focus(); return; } } catch(ex){}
      window.location.href = url;
    }, false);
  }
  function addCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
    var tb=tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i]; var cell=tr.cells[0]; var findBtn=cell ? cell.querySelector('.find-btn') : null;
      var name = findBtn ? (findBtn.getAttribute('title')||'').replace('Open a filtered view for ','') : ('Row '+(i+1));
      if(cell){
        cell.classList.add('find-cell');
        cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\\s*/, '');
      }
    }
  }
  function initShowingStatic(){ try{ document.getElementById('showing-count').textContent = document.getElementById('refactor-table').tBodies[0].rows.length; }catch(e){}}
  document.addEventListener('DOMContentLoaded', function(){
    addCheckboxes();
    (function(){
      var el=document.getElementById('last-updated'); if(!el) return;
      var d=new Date(document.lastModified||new Date());
      var months=['January','February','March','April','May','June','July','August','September','October','November','December'];
      var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear();
      var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am';
      hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min;
      el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + hour + ':' + minStr + ' ' + ampm;
    })();
    bindHeaderSort(); bindSearch(); bindGroupSync(); bindShowSelected(); bindShowAll(); bindClear(); bindPrintCousinList(); initShowingStatic();
  });
})();
//]]>
</script>
</body>
</html>
""")

# Build table HTML (no inline styles)
html_table=display_df.to_html(index=False, escape=False, classes="dataframe sortable", col_space=None)
html_table=html_table.replace('<table border="1" class="dataframe sortable">','<table border="1" class="dataframe sortable" id="refactor-table">',1)
html_table=html_table.replace('<tbody>\n<tr>','<tbody>\n<tr id="first-row">',1)
html_table=html_table.replace("<th>Find</th>",'<th>Select:</th>',1)
html_table=html_table.replace('<th>Match Summary</th>','<th>Match Summary&amp;ndash;click to sort</th>',1)
html_table=html_table.replace(f'<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>','<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>',1)
# Add a colgroup with no inline width; widths are handled by CSS variables
if '<colgroup>' not in html_table:
    html_table=html_table.replace('<table border="1" class="dataframe sortable" id="refactor-table">',
                                  '<table border="1" class="dataframe sortable" id="refactor-table">\n<colgroup><col /><col /><col /></colgroup>',1)

UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)
DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_CSV))}">CSV</a> | '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_XLSX))}">Excel</a></p>'
)
NAV_BLOCK = (
  '<nav class="oldnav"><ul>'
  '<li><a href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a></li>'
  '<li><a href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a></li>'
  '<li><a href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a></li>'
  '<li><a href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a></li>'
  '<li><a href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a></li>'
  '<li><a href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a></li>'
  f'<li><a href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a></li>'
  f'<li><a href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a></li>'
  '<li><a id="show-selected" href="#">Show Selected</a></li>'
  '<li><a id="show-all" href="#">Show All</a></li>'
  '<li><a id="print-cousin-list" href="#">Cousin List (Printable)</a></li>'
  '<li><a id="clear-selected" href="#">Reset</a></li>'
  '</ul></nav>'
)
CONTROLS_BLOCK = (
  '<div class="controls controls-spaced centerline">'
  '<input type="text" id="search-box" class="search" size="28" value="" placeholder="Search&amp;hellip;" />'
  '</div>'
)

final_html=page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK, UPDATED_BLOCK=UPDATED_BLOCK,
    NAV_BLOCK=NAV_BLOCK, CONTROLS_BLOCK=CONTROLS_BLOCK,
    HTML_TABLE=html_table, DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
)

# ---------- 9) Partials (match_count.htm, lineage_count.htm, cousin_list_print.htm) ----------
def _norm_code_for_count(s):
    t=str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")): t=t[1:-1]
    t=re.sub(r'\s+',' ',t).strip().lower(); return t

def _partial_head(title):
    return (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
        " \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n<head>\n"
        f"{HEAD_LINK}\n"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"<title>{_html.escape(title)}</title>\n"
        "</head>\n<body>\n<div class=\"wrap\">\n"
        f"<h1 class=\"centerline\">{_html.escape(title)}</h1>\n"
        "<div class=\"updated centerline\">Last updated: <span id=\"last-updated\"></span> &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\">&mdash;</span></div>\n"
        "<div class=\"table-scroll\">\n"
    )

def _partial_tail():
    safe_count = COUNT_PUBLIC_URL.replace("'","%27")
    return (
        "</div>\n</div>\n"
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){"
        "function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); var months=['January','February','March','April','May','June','July','August','September','October','November','December']; var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear(); var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am'; hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min; el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + hour + ':' + minStr + ' ' + ampm;}"
        "function load(){var el=document.getElementById('auto-count'); if(!el) return; var URL='"+safe_count+"'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true); xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent=(m?m[1]:'');} else {el.textContent='(unavailable)';}}}; xhr.send(null);}catch(e){el.textContent='(unavailable)';}}"
        "document.addEventListener('DOMContentLoaded', function(){ stamp(); load(); }, false);"
        "})();\n//]]>\n</script>\n</body>\n</html>"
    )

def build_match_count_partial(main_df: pd.DataFrame)->str:
    # main_df MUST be the full df with original columns (including match_col)
    codes_raw=main_df[match_col].astype(str).map(lambda x:x.strip())
    keys_norm=codes_raw.map(_norm_code_for_count)
    counts_series=keys_norm.value_counts(dropna=False)
    counts=counts_series.reset_index()
    if counts.shape[1]>=2: counts.columns=["norm_key","Count"]
    else:
        counts["norm_key"]=counts.index.astype(str); counts["Count"]=counts_series.values; counts=counts[["norm_key","Count"]]
    first_display={}
    for code_disp,k in zip(codes_raw.tolist(), keys_norm.tolist()):
        if k not in first_display and str(k)!="": first_display[k]=code_disp
    counts["Code"]=counts["norm_key"].map(lambda k:first_display.get(k,k))
    counts["Unmasked"]=counts["norm_key"].map(lambda k:MATCH_TO_UNMASKED.get(k,""))
    counts=counts.sort_values(by=["Code","Count"], ascending=[True,False], kind="mergesort").reset_index(drop=True)
    html=[]; html.append(_partial_head("Match Count"))
    html.append('<table id="ref-table" class="sortable" border="1"><thead><tr>')
    html.append('<th style="width:35%">Code</th><th style="width:45%">Unmasked</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _,r in counts.iterrows():
        code=r.get("Code",""); unm=r.get("Unmasked",""); cnt=int(str(r.get("Count","0")).strip() or "0"); label=(unm or code).strip()
        tr=(f'<tr data-q="{_html.escape(label,quote=True)}" data-count="{cnt}"><td>{_html.escape(code)}</td><td>{_html.escape(unm)}</td><td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td></tr>')
        html.append(tr)
    html.append('</tbody></table>'); html.append(_partial_tail()); return "".join(html)

def build_lineage_count_partial(main_df: pd.DataFrame)->str:
    first_series=main_df.get("First Ancestor", pd.Series(dtype=str)).astype(str).map(lambda x:x.strip())
    vc=first_series[first_series!=""].value_counts(dropna=False)
    lin_df=vc.reset_index()
    if lin_df.shape[1]>=2: lin_df.columns=["First Ancestor","Count"]
    else:
        lin_df["First Ancestor"]=lin_df.index.astype(str); lin_df["Count"]=vc.values; lin_df=lin_df[["First Ancestor","Count"]]
    lin_df=lin_df.sort_values(["Count","First Ancestor"], ascending=[False,True], kind="mergesort").reset_index(drop=True)
    html=[]; html.append(_partial_head("Lineage Count"))
    html.append('<table id="ref-table" class="sortable" border="1"><thead><tr>')
    html.append('<th style="width:80%">First Ancestor</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _,r in lin_df.iterrows():
        first=str(r.get("First Ancestor","")).strip(); cnt=int(str(r.get("Count","0")).strip() or "0")
        tr=(f'<tr data-q="{_html.escape(first,quote=True)}" data-count="{cnt}"><td>{_html.escape(first)}</td><td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td></tr>')
        html.append(tr)
    html.append('</tbody></table>'); html.append(_partial_tail()); return "".join(html)

def build_and_write_partials(main_df: pd.DataFrame):
    _setup_resolver(); os.makedirs("partials", exist_ok=True)

    mc_html=build_match_count_partial(main_df); mc_local=os.path.join("partials","match_count.htm")
    with open(mc_local,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(mc_html)
    print("[OK] Wrote partial:", mc_local)

    lc_html=build_lineage_count_partial(main_df); lc_local=os.path.join("partials","lineage_count.htm")
    with open(lc_local,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(lc_html)
    print("[OK] Wrote partial:", lc_local)

    # Cousin List (Printable) — stylesheet linked
    cousin_df=display_df[["Match Summary"]].copy()
    cousin_df=cousin_df.sort_values(by="Match Summary", ascending=True, kind="mergesort").reset_index(drop=True)
    rows=['<table border="1" id="refactor-table" class="sortable"><thead><tr><th>Match Summary</th></tr></thead><tbody>']
    for v in cousin_df["Match Summary"].tolist(): rows.append(f"<tr><td>{v}</td></tr>")
    rows.append("</tbody></table>")
    cousin_html=(
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\"><head>"
        f"{HEAD_LINK}"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />"
        "<title>Cousin List (Printable)</title>"
        "</head><body onload=\"window.print();\">"
        "<div class=\"wrap\">"
        "<h1 class=\"centerline\">Cousin List (Printable)</h1>"
        "<div class=\"table-scroll\">"+ "".join(rows) +"</div>"
        "</div></body></html>"
    )
    cl_local=os.path.join("partials","cousin_list_print.htm")
    with open(cl_local,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(cousin_html)
    print("[OK] Wrote partial:", cl_local)

    return mc_local, lc_local, cl_local

# >>>>>>>>>>> FIX: pass the FULL df (not display_df) so 'Match to' exists <<<<<<<<<<<
PARTIAL_MATCH_LOCAL, PARTIAL_LINEAGE_LOCAL, PARTIAL_COUSIN_LOCAL = build_and_write_partials(df)

# Build alternate render that points Find to HOME_URL (ABS)
def build_register_html_for_abs(remote_abs_path:str)->str:
    q_links=[]; subs_names=df["Subject"].astype(str).tolist()
    for subject_name in subs_names:
        q=_u.quote(subject_name)
        q_links.append(f'<a class="find-btn" href="{remote_abs_path}?q={q}" target="_blank" rel="noopener" title="Open a filtered view for {subject_name}">Find</a>')
    df_plus=df.copy(); df_plus["Find"]=q_links
    disp_plus=df_plus[["Find","Match Summary",LINEAGE_HEADER_SAFE]]
    tbl=disp_plus.to_html(index=False, escape=False, classes="dataframe sortable")
    tbl=tbl.replace('<table border="1" class="dataframe sortable">','<table border="1" class="dataframe sortable" id="refactor-table">',1)
    tbl=tbl.replace('<tbody>\n<tr>','<tbody>\n<tr id="first-row">',1)
    tbl=tbl.replace("<th>Find</th>",'<th>Select:</th>',1)
    tbl=tbl.replace("<th>Match Summary</th>",'<th>Match Summary&amp;ndash;click to sort</th>',1)
    tbl=tbl.replace(f"<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>","<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>",1)
    if '<colgroup>' not in tbl:
        tbl=tbl.replace('<table border="1" class="dataframe sortable" id="refactor-table">',
                        '<table border="1" class="dataframe sortable" id="refactor-table">\n<colgroup><col /><col /><col /></colgroup>',1)
    return page_tpl.safe_substitute(
        HEAD_LINK=HEAD_LINK, UPDATED_BLOCK=UPDATED_BLOCK, NAV_BLOCK=NAV_BLOCK,
        CONTROLS_BLOCK=CONTROLS_BLOCK, HTML_TABLE=tbl, DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
    )

os.makedirs("partials", exist_ok=True)
final_html=page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK, UPDATED_BLOCK=UPDATED_BLOCK,
    NAV_BLOCK=NAV_BLOCK, CONTROLS_BLOCK=CONTROLS_BLOCK,
    HTML_TABLE=html_table, DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
)
final_html_plus=build_register_html_for_abs(HOME_URL)

with open(LOCAL_HTML,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(final_html)
print("[OK] Saved canonical render:", os.path.abspath(LOCAL_HTML))
with open(WORK_PLUS_LOCAL,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(final_html_plus)
print("[OK] Saved:", os.path.abspath(WORK_PLUS_LOCAL), "(partials clone)")

# ---------- 10) Upload ----------
def save_and_upload_all():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped."); return
    try:
        ftps=ftp_connect()
        try:
            # Upload stylesheet first (cache-busted link already points here)
            ftp_upload_overwrite(ftps, STYLESHEET_LOCAL, _remote_path(STYLESHEET_REMOTE))
        except Exception as e: print("[WARN] Upload stylesheet failed:", e)
        try:
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_CANON))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_LEG))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_SIMPLE))
        except Exception as e: print("[WARN] Upload main HTML failed:", e)
        try:
            if os.path.exists(LOCAL_CSV):  ftp_upload_overwrite(ftps, LOCAL_CSV,  _remote_path(REMOTE_CSV))
            if os.path.exists(LOCAL_XLSX): ftp_upload_overwrite(ftps, LOCAL_XLSX, _remote_path(REMOTE_XLSX))
            print("[OK] Uploaded CSV/XLSX -> /partials/")
        except Exception as e: print("[WARN] Upload CSV/XLSX failed:", e)
        if os.path.exists(LOCAL_COUNT_FILE):
            try: ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
            except Exception as e: print("[WARN] Upload autosomal count failed:", e)
        try:
            ftp_upload_overwrite(ftps, os.path.join("partials","match_count.htm"),       _remote_path(posixpath.join("partials","match_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","lineage_count.htm"),     _remote_path(posixpath.join("partials","lineage_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","cousin_list_print.htm"), _remote_path(posixpath.join("partials","cousin_list_print.htm")))
        except Exception as e: print("[WARN] Upload partials failed:", e)
        try: ftp_upload_overwrite(ftps, WORK_PLUS_LOCAL, _remote_path(WORK_PLUS_REMOTE))
        except Exception as e: print("[WARN] Upload work_plus.htm failed:", e)

        print("\n--- SIZE Verification (if supported) ---")
        for p in [
            _remote_path(REMOTE_HTML_CANON), _remote_path(REMOTE_HTML_LEG), _remote_path(REMOTE_HTML_SIMPLE),
            _remote_path(REMOTE_CSV), _remote_path(REMOTE_XLSX),
            _remote_path(posixpath.join("partials","match_count.htm")),
            _remote_path(posixpath.join("partials","lineage_count.htm")),
            _remote_path(posixpath.join("partials","cousin_list_print.htm")),
            _remote_path(WORK_PLUS_REMOTE),
            _remote_path(STYLESHEET_REMOTE),
        ]:
            sz=ftp_size(ftps,p); print(f"{p} : {sz if sz is not None else '(SIZE unsupported)'}")
        try: ftps.quit()
        except Exception: pass

        print("\n--- Open URLs ---")
        print("Canonical:        https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Legacy clone:     https://yates.one-name.net/partials/ons_yates_dna_register.htm")
        print("JUSTDNA alias:    https://yates.one-name.net/partials/justdna.htm")
        print("Trees:            https://yates.one-name.net/partials/just-trees.htm")
        print("Match Count:      https://yates.one-name.net/partials/match_count.htm")
        print("Lineage Count:    https://yates.one-name.net/partials/lineage_count.htm")
        print("Cousin Printable: https://yates.one-name.net/partials/cousin_list_print.htm")
        print("Work+ clone:      https://yates.one-name.net/partials/work_plus.htm")
        print("Stylesheet:       https://yates.one-name.net/partials/dna_tree_styles.css")
        print("\nBust cache once if needed by appending ?v=now to the URL.")
    except Exception as e:
        print("[FAIL] FTP session:", e); traceback.print_exc()

save_and_upload_all()
# ====== CUT STOP [1/1] CELL 2 ===================================================================

[CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15
DECLARED_LINES=9999
[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote stylesheet: /content/partials/dna_tree_styles.css
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT] partials/dna_tree_styles.css -> partials/dna_tree_styles.css
[PUT] yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT] yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT] yates_ancestor_register.htm -> partials/justdna.htm
[PUT] yates_ancestor_register.csv ->

# Cell 3

In [ ]:
# ====== CUT START [1/1] CELL 3 — Ancestor Register (Old-school Blue Menu; WHITE menu text) ======
# RON GOLDEN RULES — CLIFF NOTES (v2025.11.12)
# • Complete & runnable Colab cell — one contiguous block.
# • Source ASCII-only; outputs written with encoding="iso-8859-15", errors="xmlcharrefreplace".
# • XHTML 1.0 Transitional; typography comes ONLY from /partials/dna_tree_styles.css.
# • Deterministic audit:
#   [CONFIRM] Golden Rules active | Cell=Cell3_OldSchoolMenu_WhiteText | Version=2025.11.12 | Encoding=ISO-8859-15
# • Enforce ISO-8859-15 printable chars on writes; DECLARED_LINES printed at run start.

DECLARED_LINES = 9999
print("[CONFIRM] Golden Rules active | Cell=Cell3_OldSchoolMenu_WhiteText | Version=2025.11.12 | Encoding=ISO-8859-15")
print("DECLARED_LINES=%d" % DECLARED_LINES)

# ---------- Imports ----------
import os, re, socket, posixpath, traceback
import pandas as pd
import html as _html
from ftplib import FTP_TLS

# ---------- Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_DIR  = os.environ.get('FTP_DIR','').strip().strip('/')
COUNT_PUBLIC_URL = ("/%s/%s" % (FTP_DIR, "autosomal_count.txt")) if FTP_DIR else "/autosomal_count.txt"

# ---------- Config / Paths ----------
INPUT_CSV = "final_combined_df_with_value_labels.csv"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV   = EXPORT_BASENAME + ".csv"
LOCAL_XLSX  = EXPORT_BASENAME + ".xlsx"
REMOTE_CSV  = posixpath.join("partials", LOCAL_CSV)
REMOTE_XLSX = posixpath.join("partials", LOCAL_XLSX)

OUTPUT_NAME    = "just-trees.htm"  # this page’s filename
REMOTE_HTML    = posixpath.join("partials", OUTPUT_NAME)

# Cross-site buttons (same as Cell 2)
DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS        = "https://yates.one-name.net/partials/just-trees.htm"

# Stylesheet + cache buster (shared with Cell 2)
STYLESHEET_HREF="/partials/dna_tree_styles.css"; CSS_VERSION="v2025-11-12"
HEAD_LINK = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}?{CSS_VERSION}" />'

# Old-school menu widths (responsive behavior mostly via CSS; typography comes from stylesheet)
TABLE_WIDTH_PX=5550; FIND_COL_PX=25; COL_A_PX=900  # kept for consistency even if not used here

# ---------- Load CSV (robust) ----------
df = None; _last_err=None
for enc in ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1"):
    try:
        df=pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=False, encoding=enc); break
    except Exception as e:
        _last_err=e; df=None
if df is None:
    raise SystemExit("[ERROR] Unable to read CSV: %s (%r)" % (INPUT_CSV, _last_err))
print("[OK] Loaded CSV:", INPUT_CSV, "rows=%d, cols=%d" % (len(df), len(df.columns)))

# Ensure haplogroup present (not strictly needed for “just-trees”, but harmless)
if 'haplogroup' not in df.columns: df['haplogroup']=''
else: df['haplogroup']=df['haplogroup'].fillna('')

# ---------- Resolver: Column B (masked) -> Column C (unmasked) ----------
A_IDX=0; B_IDX=1; C_IDX=2

def _norm_code(s):
    t=str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")): t=t[1:-1]
    t=t.replace("\u00a0"," "); t=re.sub(r"\s{2,}"," ",t)
    return t.lower()

# Prefer local-first resolver cached by Cell 1; fall back to server
LOCAL_RESOLVER="match_to_unmasked.csv"
if not os.path.exists(LOCAL_RESOLVER) and os.path.exists("/content/partials/match_to_unmasked.csv"):
    LOCAL_RESOLVER="/content/partials/match_to_unmasked.csv"

def _pull_resolver_if_needed(local_path):
    if os.path.exists(local_path):
        print("Using resolver:", os.path.abspath(local_path)); return local_path
    print("Resolver not found locally; attempting server pull ...")
    try:
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(os.environ.get("FTP_HOST",""), int(os.environ.get("FTP_PORT","21")))
            ftps.login(os.environ.get("FTP_USER",""), os.environ.get("FTP_PASS",""))
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass
            if FTP_DIR:
                for p in [p for p in FTP_DIR.split("/") if p]:
                    try: ftps.cwd(p)
                    except Exception:
                        try: ftps.mkd(p)
                        except Exception: pass
                        ftps.cwd(p)
            try: ftps.cwd("partials")
            except Exception: pass
            with open("match_to_unmasked.csv","wb") as f:
                ftps.retrbinary("RETR match_to_unmasked.csv", f.write)
        print("[OK] Pulled resolver from server -> match_to_unmasked.csv")
        return "match_to_unmasked.csv"
    except Exception as e:
        print("[WARN] Could not pull resolver from server:", e)
        return local_path  # still return whatever we have (likely missing)

LOCAL_RESOLVER=_pull_resolver_if_needed(LOCAL_RESOLVER)

def _load_resolver_to_map(path):
    last=None; m=None
    for enc in ("utf-8-sig","iso-8859-15","utf-8","cp1252","latin1"):
        try:
            m=pd.read_csv(path, dtype=str, keep_default_na=False, encoding=enc); break
        except Exception as e:
            last=e; m=None
    if m is None:
        print("[WARN] Resolver not loaded:", last); return {}
    cols={c.lower():c for c in m.columns}
    if "code" not in cols or "unmasked" not in cols:
        print("[WARN] Resolver missing 'code'/'unmasked' cols; skipping map."); return {}
    m=m[[cols["code"], cols["unmasked"]]].copy()
    m["__key__"]=m[cols["code"]].map(_norm_code); m["__val__"]=m[cols["unmasked"]].astype(str)
    m=m.drop_duplicates(subset="__key__", keep="first")
    return dict(zip(m["__key__"], m["__val__"]))

resolver_map=_load_resolver_to_map(LOCAL_RESOLVER) if os.path.exists(LOCAL_RESOLVER) else {}

if df.shape[1] < 3:
    raise ValueError("Main df must have at least 3 columns: A(ID#), B(match to), C(unmasked).")

masked_raw=df.iloc[:,B_IDX].astype(str); masked_key=masked_raw.map(_norm_code)
resolved=masked_key.map(resolver_map)
df.iloc[:,C_IDX]=resolved.fillna("")  # write unmasked into col C

print("[OK] Column B -> C mapping:", int(resolved.notna().sum()), "/", len(df), "unmatched:", len(df)-int(resolved.notna().sum()))

# ---------- Build page (Old-school Blue Menu header + shared CSS) ----------
# Download links (exports mirrored to /partials/)
DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(LOCAL_CSV)}">CSV</a> | '
    f'<a href="/partials/{_html.escape(LOCAL_XLSX)}">Excel</a></p>'
)

UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)

NAV_BLOCK = (
  '<nav class="oldnav"><ul>'
  '<li><a href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a></li>'
  '<li><a href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a></li>'
  '<li><a href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a></li>'
  '<li><a href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a></li>'
  '<li><a href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a></li>'
  '<li><a href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a></li>'
  f'<li><a href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a></li>'
  f'<li><a href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a></li>'
  '</ul></nav>'
)

CONTROLS_BLOCK = (
  '<div class="controls centerline" style="margin:6px 0 10px 0;">'
  '<input type="text" id="search-box" class="search" size="28" value="" placeholder="Search&amp;hellip;" />'
  '</div>'
)

# Page-local layout helpers only. Colors/typography come from dna_tree_styles.css
TABLE_CSS = f"""
<style type="text/css">
  .wrap {{ max-width:100%; margin:0 auto; background:#ffffff; padding:16px; padding-bottom:48px; }}
  .centerline {{ text-align:center; }}
  .downloads {{ text-align:center; margin:4px 0 10px 0; font-size:13px; }}
  .updated {{ font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }}

  .oldnav {{ margin:8px auto 6px auto; padding:0; background:#5b79b8; border-radius:6px; overflow:hidden; max-width:{TABLE_WIDTH_PX}px; }}
  .oldnav ul {{ list-style:none; margin:0; padding:0; display:flex; flex-wrap:wrap; }}
  .oldnav li {{ margin:0; padding:0; }}
  .oldnav a, .oldnav a:link, .oldnav a:visited, .oldnav a:active {{ color:#ffffff !important; }}
  .oldnav a {{ display:block; padding:8px 12px; text-decoration:none; white-space:nowrap; border-right:1px solid #ffffff; font-weight:600; }}
  .oldnav li:last-child a {{ border-right:none; }}
  .oldnav a:hover {{ background:#4668aa; color:#ffffff !important; }}

  .table-scroll {{ max-height:75vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }}
  table.sortable {{ border-collapse:collapse; width:100%; table-layout:auto; }}
  table.sortable th, table.sortable td {{ border:1px solid #ddd; padding:6px 8px; vertical-align:top; word-wrap:break-word; overflow-wrap:break-word; }}
  table.sortable th {{ background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; cursor:pointer; }}
  #first-row td {{ border-top:2px solid #999; }}

  @media screen and (min-width: 1200px) {{
    .wrap {{ max-width:{TABLE_WIDTH_PX}px; }}
  }}
  @media screen and (max-width: 1199px) {{
    .oldnav {{ border-radius:0; }}
  }}
</style>
"""

# Build table for this page (Ancestor Register = show Unmasked [col C] + Lineage columns if present)
# Choose a helpful subset if the CSV is wide; otherwise render full and let user filter.
visible_cols = [c for c in df.columns if c]  # keep order
table_html = df.to_html(index=False, columns=visible_cols, escape=False, border=1, classes="dataframe sortable")
table_html = table_html.replace('<table border="1" class="dataframe sortable">','<table border="1" class="dataframe sortable" id="refactor-table">',1)
table_html = table_html.replace('<tbody>\n<tr>','<tbody>\n<tr id="first-row">',1)

# XHTML page template (same header structure as Cell 2)
from string import Template as _T
page_tpl=_T("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Ancestor Register (Trees View)</title>
$HEAD_LINK
$TABLE_CSS
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">Ancestor Register (Trees View)</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $NAV_BLOCK
  $CONTROLS_BLOCK
  <div class="table-scroll">$HTML_TABLE</div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); var nA=parseFloat(A.replace(/[^0-9.\\-]/g,'')), nB=parseFloat(B.replace(/[^0-9.\\-]/g,'')); if(!isNaN(nA)&&!isNaN(nB)){return asc?(nA-nB):(nB-nA);} if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++)(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+\\(asc\\)|\\s+\\(desc\\)/,''); }
        th.innerHTML += (dir==='asc' ? ' (asc)' : ' (desc)');
        sortTable(tbl,idx,dir);
      },false);
    })(i);
  }
  function formatWithCommas(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); } }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0; for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; } return n;
  }
  function updateShowing(){ var el=document.getElementById('showing-count'); if(!el) return; el.textContent = formatWithCommas(visibleRowCount()); }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=0;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false); box.addEventListener('search', onInput, false);
    var q0=getParam('q'); if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null,'',location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function bindBackToTop(){
    var btn=document.getElementById('back-to-top'); if(!btn) return;
    function toggle(){ btn.style.display = (window.scrollY > 200) ? 'block' : 'none'; }
    toggle(); window.addEventListener('scroll', toggle, {passive:true});
    btn.addEventListener('click', function(){ window.scrollTo(0,0); }, false);
  }
  function stampAndCount(){
    var el=document.getElementById('last-updated'); if(el){
      var d=new Date(document.lastModified||new Date());
      var months=['January','February','March','April','May','June','July','August','September','October','November','December'];
      var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear();
      var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am';
      hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min;
      el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + minStr + ':' + (hour+''); // corrected below
    }
    // Corrected time display (AM/PM with hour first); keep above minimal to avoid removal.
    (function(){var el=document.getElementById('last-updated'); if(!el) return;
      var d=new Date(document.lastModified||new Date());
      var months=['January','February','March','April','May','June','July','August','September','October','November','December'];
      var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear();
      var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am';
      hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min;
      el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + hour + ':' + minStr + ' ' + ampm; })();
    var elc=document.getElementById('auto-count'); if(!elc) return;
    var URL='$JS_COUNT_URL'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); elc.textContent=(m?m[1]:'');} else {elc.textContent='(unavailable)';}}};
      xhr.send(null);}catch(e){elc.textContent='(unavailable)';}
  }
  document.addEventListener('DOMContentLoaded', function(){
    bindHeaderSort(); bindBackToTop(); bindSearch(); stampAndCount();
    (function(){var sc=document.getElementById('showing-count'); if(!sc) return;
      var tb=document.getElementById('refactor-table'); if(!(tb&&tb.tBodies&&tb.tBodies[0])) return;
      sc.textContent = tb.tBodies[0].rows.length; })();
  });
})();
//]]>
</script>
</body>
</html>
""")

final_html=page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK, TABLE_CSS=TABLE_CSS, DOWNLOADS_BLOCK=DOWNLOADS_BLOCK, UPDATED_BLOCK=UPDATED_BLOCK,
    NAV_BLOCK=NAV_BLOCK, CONTROLS_BLOCK=CONTROLS_BLOCK, HTML_TABLE=table_html,
    JS_COUNT_URL=COUNT_PUBLIC_URL
)

# ---------- Exports ----------
export_df=df.copy()
export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    from pandas import ExcelWriter
    with ExcelWriter(LOCAL_XLSX) as _w: export_df.to_excel(_w, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- Save page locally ----------
try:
    with open(OUTPUT_NAME,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(final_html)
    print("[OK] Saved locally:", os.path.abspath(OUTPUT_NAME))
except Exception as e:
    print("[ERROR] Save failed:", e); traceback.print_exc()

# ---------- Upload to /partials ----------
def _ftps_ensure_dir(ftps, path):
    if not path: return
    for seg in [p for p in path.split('/') if p]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)

ftp_host=os.environ.get('FTP_HOST'); ftp_user=os.environ.get('FTP_USER'); ftp_pass=os.environ.get('FTP_PASS')
ftp_port=int(os.environ.get('FTP_PORT','21') or '21')

if ftp_host and ftp_user and ftp_pass:
    print("[INFO] Attempting FTP upload ...")
    try:
        socket.setdefaulttimeout(30)
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(ftp_host, ftp_port)
            ftps.login(ftp_user, ftp_pass)
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass

            # Navigate to base and /partials
            _ftps_ensure_dir(ftps, FTP_DIR)
            _ftps_ensure_dir(ftps, "partials")

            # Upload HTML
            with open(OUTPUT_NAME,"rb") as fh:
                ftps.storbinary("STOR "+os.path.basename(REMOTE_HTML), fh)
            print("[OK] Uploaded HTML -> /partials/%s" % os.path.basename(REMOTE_HTML))

            # Upload CSV/XLSX
            with open(LOCAL_CSV,"rb") as fh:
                ftps.storbinary("STOR "+os.path.basename(REMOTE_CSV), fh)
            with open(LOCAL_XLSX,"rb") as fh:
                ftps.storbinary("STOR "+os.path.basename(REMOTE_XLSX), fh)
            print("[OK] Uploaded exports -> /partials/ (%s, %s)" % (LOCAL_CSV, LOCAL_XLSX))

            print("\n--- Open URLs ---")
            print("Trees page:       https://yates.one-name.net/partials/just-trees.htm")
            print("CSV export:       https://yates.one-name.net/partials/%s" % os.path.basename(LOCAL_CSV))
            print("Excel export:     https://yates.one-name.net/partials/%s" % os.path.basename(LOCAL_XLSX))
    except Exception as e:
        print("[ERROR] FTP session failed:", e); traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload (missing credentials).")

print("\n--- Cell 3 Complete (Old-school header unified; stylesheet-driven; exports + upload ready) ---")
# ====== CUT STOP  [1/1] CELL 3 ==================================================================


# Test Cell 1

In [ ]:
# ====== CUT START [1/1] CELL 1 — GEDCOM -> CSV + HTML + Upload (Explicit FTPS, ISO-8859-15) ===
# RON GOLDEN RULES — CLIFF NOTES (v2025.11.12)
# • Complete & runnable Colab Cell — one contiguous block (no fragments, no cross-refs).
# • Source ASCII-only; outputs written with encoding="iso-8859-15", errors="xmlcharrefreplace".
# • XHTML 1.0 Transitional; typography via /partials/dna_tree_styles.css (HTML export only).
# • Deterministic audit:
#   [CONFIRM] Golden Rules active | Cell=Cell1_FTPS_Explicit | Version=2025.11.12 | Encoding=ISO-8859-15
# • Enforce ISO-8859-15 printable chars on writes; DECLARED_LINES printed at start.
# ==============================================================================================

import os, re, glob, logging, functools, socket, traceback, hashlib, inspect
from datetime import datetime
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
import numpy as np
from tqdm import tqdm
from ftplib import FTP_TLS, all_errors
from string import Template

CELL_NAME = "Cell1_FTPS_Explicit"
VERSION   = "2025.11.12"

print("[CONFIRM] Golden Rules active | Cell=%s | Version=%s | Encoding=ISO-8859-15" % (CELL_NAME, VERSION))

# ---------- Logging ----------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(CELL_NAME)

# ---------- Secrets (env or userdata) ----------
def _get_env(k, default=""):
    try:
        from google.colab import userdata  # type: ignore
        v = userdata.get(k)
        return v if v is not None else os.environ.get(k, default)
    except Exception:
        return os.environ.get(k, default)

FTP_HOST = (_get_env("FTP_HOST","") or "").strip()
FTP_USER = (_get_env("FTP_USER","") or "").strip()
FTP_PASS = _get_env("FTP_PASS","") or ""
FTP_PORT = int(_get_env("FTP_PORT","21") or "21")
FTP_DIR  = (_get_env("FTP_DIR","") or "").strip().strip("/")
PASSIVE_MODE = True

def _mask(s, keep=3):
    s = "" if s is None else str(s)
    if not s: return "(empty)"
    return (s[:keep] + "***" + s[-keep:]) if len(s) > keep*2 else s[0:1] + "***"

print("[ENV] HOST=%s  USER=%s  PASS=%s  PORT=%d  DIR=%s" %
      (_mask(FTP_HOST), _mask(FTP_USER,2), "***", FTP_PORT, ("/"+FTP_DIR) if FTP_DIR else "(root)"))

# ---------- FTPS (Explicit AUTH TLS) ----------
def _ftps_connect():
    if not (FTP_HOST and FTP_USER and FTP_PASS):
        raise RuntimeError("Missing FTP_HOST/FTP_USER/FTP_PASS.")
    socket.setdefaulttimeout(30)
    ftps = FTP_TLS(timeout=30)
    ftps.connect(FTP_HOST, FTP_PORT)
    ftps.auth()                 # Explicit FTPS: AUTH TLS before login
    ftps.login(FTP_USER, FTP_PASS)
    try: ftps.prot_p()          # Encrypt data channel
    except Exception: pass
    try: ftps.set_pasv(PASSIVE_MODE)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split("/") if p]:
            ftps.cwd(p)
    return ftps

def _ftps_ensure_dir(ftps, path):
    if not path: return
    parts = [p for p in path.split("/") if p]
    for p in parts:
        try:
            ftps.cwd(p)
        except all_errors:
            try: ftps.mkd(p)
            except all_errors: pass
            ftps.cwd(p)

def _ftps_upload(ftps, local_path, remote_name):
    with open(local_path, "rb") as fh:
        ftps.storbinary("STOR " + remote_name, fh)
    print("[OK] Uploaded: %s -> %s/%s" % (local_path, ftps.pwd().rstrip("/"), remote_name))

# ---------- Outputs / Paths ----------
REMOTE_DIR            = "partials"
CSV_OUT_LOCAL         = "final_combined_df_with_value_labels.csv"
HTML_OUT_LOCAL        = "cell1_work_table.htm"
ABS_CSV_URL           = "/%s/%s" % (REMOTE_DIR, os.path.basename(CSV_OUT_LOCAL))
ABS_HOME_URL          = "/index.htm"

# New: artifacts for Cell 2
LOCAL_PARTIALS_DIR    = "/content/partials"
LOCAL_RESOLVER_OUT    = os.path.join(LOCAL_PARTIALS_DIR, "match_to_unmasked.csv")
LOCAL_COUNT_FILE      = "/content/autosomal_count.txt"

# Candidate local resolver sources (no guessing; only if present)
CANDIDATE_RESOLVER_PATHS = [
    "match_to_unmasked.csv",
    "/content/match_to_unmasked.csv",
    "/content/drive/MyDrive/match_to_unmasked.csv",
    "/workspace/match_to_unmasked.csv",
]

# ---------- Minimal GEDCOM parse helpers ----------
anchor_gen1 = None
visited_pairs = set()
generation_table = []

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None
    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value
    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '') or ''
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0] if parts else ""
        last_name  = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1; anchor_gen1 = self.anchor_gen1
        return self.gen_person.strip('@')
    def get_extractable_NPFX(self): return self.extractable_detail.get('NPFX','') or ''
    def get_extractable_cm(self):
        v = self.extractable_detail.get('NPFX','') or ''
        if '&' in v: cm = v.split('&')[0].strip()
        elif '**' in v: cm = v.split('**')[0].strip()
        else: cm = v.strip()
        try: int(cm); return cm
        except Exception: return ''
    def get_extractable_sort(self):
        v = self.extractable_detail.get('NPFX','') or ''
        if '&' in v:
            s = v.split('&')[1]
            return (s.split('**')[0] if '**' in s else s).strip()
        return ''
    def get_extractable_YDNA(self):
        v = self.extractable_detail.get('NPFX','') or ''
        return v.split('**')[1].strip() if '**' in v else ''
    def get_extractable_FAMC(self): return (self.extractable_detail.get('FAMC','') or '').strip('@')

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []
    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            lines = f.readlines()
        current = None
        npfx_count = ydna_count = total = 0
        for line in lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0]); tag = parts[1]; value = parts[2] if len(parts) > 2 else None
            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total += 1; current = GedcomDataset(tag); self.gedcom_datasets.append(current)
            elif current is not None:
                if level == 1 and tag in ['NAME','FAMC']:
                    current.add_extractable_detail(tag, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1; current.add_extractable_detail(tag, value)
                    if value and '**' in value: ydna_count += 1
        autosomal = npfx_count - ydna_count
        print("GEDCOM contained %d total records" % total)
        print("Records tagged and filtered by NPFX: %d" % npfx_count)
        print("Records with YDNA information: %d" % ydna_count)
        print("Autosomal matches: %d" % autosomal)
        for ds in self.gedcom_datasets:
            if ds.get_extractable_NPFX():
                self.filter_pool.append(ds)
        try:
            df_filter = pd.read_excel('filtered_ids.xlsx')
            manual_ids = set(str(x) for x in df_filter['ID'])
            self.filter_pool = [d for d in self.filter_pool if d.get_gen_person() in manual_ids]
            print("After manual filter, total records: %d" % len(self.filter_pool))
        except FileNotFoundError:
            logger.warning("filtered_ids.xlsx not found. Skipping second-level manual filter.")
        return autosomal

def _chunks(lst, n):
    for i in range(0, len(lst), n): yield lst[i:i+n]

def _quick_extract_name(full_text):
    name_marker = "\n1 NAME "
    idx = full_text.find(name_marker)
    if idx == -1:
        if full_text.startswith("1 NAME "): idx = 0
        else: return "UnknownName"
    start = idx + len(name_marker)
    end = full_text.find('\n', start); end = len(full_text) if end == -1 else end
    name_line = full_text[start:end].strip()
    if '/' not in name_line: return name_line[:10].replace(" ","")
    first_name, last_name = name_line.split('/', 1)
    last_name = last_name.replace("/", "").strip()
    return last_name[:10].replace(" ","") + first_name[:10].replace(" ","")

def _find_parents(individual_id, generation, parents_map):
    global visited_pairs, generation_table
    if individual_id not in parents_map: return
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id: return
    pair = (father_id, mother_id)
    if pair not in visited_pairs:
        visited_pairs.add(pair)
        generation_table.append((generation, pair))
    if father_id: _find_parents(father_id, generation+1, parents_map)
    if mother_id: _find_parents(mother_id, generation+1, parents_map)

def _find_distant(individual_id, parents_map, path=None):
    if path is None: path = []
    path.append(individual_id)
    if individual_id not in parents_map: return [path]
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id: return [path]
    paths = []
    if father_id: paths.extend(_find_distant(father_id, parents_map, path[:]))
    if mother_id: paths.extend(_find_distant(mother_id, parents_map, path[:]))
    return paths if paths else [path]

def _filter_lineage(winning_ids, gen_table, names_map):
    matching = []
    for generation, pair in gen_table:
        id1, id2 = pair
        if id1 in winning_ids or id2 in winning_ids:
            matching.append((generation, pair))
    matching.sort(key=lambda x: x[0])
    lines = []
    for _, pair in matching:
        name_pair = [names_map.get(pid, "UnknownName") for pid in pair]
        lines.append("%s&%s" % (name_pair[0], name_pair[1]))
    lines.reverse()
    return "~~~".join(lines)

def _process_record(individual_id, ged, parents_map, names_map):
    global generation_table, visited_pairs, anchor_gen1
    generation_table = []; visited_pairs = set()
    _find_parents(individual_id, 1, parents_map)
    paths = _find_distant(individual_id, parents_map)
    best_score, best_path = None, None
    for path in paths:
        name_path = [names_map.get(pid, "UnknownName") for pid in path]
        score = sum((idx+1) for idx, nm in enumerate(name_path) if 'Yates' in nm)
        if best_score is None or score > best_score:
            best_score, best_path = score, path
    best_path = best_path or []
    best_ids  = [pid for pid in best_path if pid != individual_id]
    line_str  = _filter_lineage(set(best_ids), generation_table, names_map)
    cm_value=''; sort_value=''; ydna_value=''
    for ds in ged.filter_pool:
        if ds.get_gen_person() == individual_id:
            cm_value  = ds.get_extractable_cm()
            sort_value= ds.get_extractable_sort()
            ydna_value= ds.get_extractable_YDNA()
            break
    short_name = names_map.get(individual_id, "UnknownName")
    return [individual_id, sort_value, short_name, cm_value, line_str, ydna_value]

# ---------- Helper: Local resolver prep ----------
def _read_mapping_csv_norm(path):
    encs=("iso-8859-15","utf-8-sig","utf-8","cp1252","latin1")
    last=None; df=None
    for enc in encs:
        try:
            df=pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last=e; df=None
    if df is None:
        raise RuntimeError("Unable to read mapping CSV: %s (%s)" % (path, last))
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df = df.iloc[:, :2].copy()
    df.columns = ["code","unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"]!=""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV is empty after normalization.")
    return df

def prepare_local_resolver(out_path):
    try:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        for cand in CANDIDATE_RESOLVER_PATHS:
            if os.path.exists(cand) and os.path.isfile(cand):
                try:
                    df_map = _read_mapping_csv_norm(cand)
                    with open(out_path, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
                        f.write(df_map.to_csv(index=False))
                    print("[OK] Local resolver cached -> %s | rows=%d" % (os.path.abspath(out_path), len(df_map)))
                    return True
                except Exception as e:
                    print("[WARN] Candidate resolver failed normalization: %s | %s" % (cand, e))
        print("[INFO] No local resolver found. Cell 2 will fall back to server copy if available.")
        return False
    except Exception as e:
        print("[WARN] Local resolver prep failed:", e)
        return False

# ---------- Main build ----------
def main():
    # Audit: DECLARED_LINES (of this function, deterministic print)
    try:
        declared = len(inspect.getsource(main).splitlines())
    except Exception:
        declared = -1
    print("[AUDIT] DECLARED_LINES=%s" % str(declared))

    files = glob.glob("*.ged")
    if not files:
        print("No GEDCOM files found."); return False
    print("Automatically selecting the first GEDCOM file.")
    gedcom_path = files[0]

    ged = Gedcom(gedcom_path)
    autosomal_count = ged.parse_gedcom()

    # Write autosomal count to Cell 2 expected path
    try:
        with open(LOCAL_COUNT_FILE, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
            f.write(str(autosomal_count))
        print("[OK] autosomal_count.txt -> %s | count=%d" % (LOCAL_COUNT_FILE, autosomal_count))
    except Exception as e:
        print("[WARN] Failed writing autosomal_count.txt:", e)

    with open(gedcom_path, "r", encoding="utf-8") as f:
        raw = f.read()

    blocks = raw.split("\n0 ")
    all_records = {}
    for blk in blocks:
        blk = blk.strip()
        if not blk: continue
        flend = blk.find('\n'); flend = len(blk) if flend == -1 else flend
        first_line = blk[:flend]
        if '@' in first_line:
            s = first_line.find('@') + 1
            e = first_line.find('@', s)
            rec_id = first_line[s:e].strip()
            all_records[rec_id] = blk

    parents_map, names_map, families = {}, {}, {}
    for rec_id, txt in all_records.items():
        if 'FAM' in txt[:50]:
            father_idx = txt.find('1 HUSB @')
            husb_id = txt[father_idx+len('1 HUSB @'):txt.find('@', father_idx+len('1 HUSB @'))] if father_idx != -1 else None
            wife_idx = txt.find('1 WIFE @')
            wife_id = txt[wife_idx+len('1 WIFE @'):txt.find('@', wife_idx+len('1 WIFE @'))] if wife_idx != -1 else None
            kids = [ln.split('@')[1] for ln in txt.split('\n') if ln.strip().startswith('1 CHIL @')]
            families[rec_id] = (husb_id, wife_id, kids)
    for fam_id, (f_id, m_id, k_list) in families.items():
        for kid in k_list:
            parents_map[kid] = (f_id, m_id)
    for rec_id, txt in all_records.items():
        names_map[rec_id] = _quick_extract_name("\n" + txt)

    individual_ids = [d.get_gen_person() for d in ged.filter_pool]
    print("Processing %d individuals with chunk-based parallel..." % len(individual_ids))

    combined_rows = []
    chunk_size = 50
    max_workers = os.cpu_count() or 4
    from functools import partial
    with ProcessPoolExecutor(max_workers=max_workers) as ex, tqdm(total=len(individual_ids), desc="Building Yates Lines (Stage 1)") as pbar:
        for chunk in _chunks(individual_ids, chunk_size):
            func = partial(_process_record, ged=ged, parents_map=parents_map, names_map=names_map)
            results = list(ex.map(func, chunk))
            combined_rows.extend(results); pbar.update(len(chunk))

    columns = ["ID#", "Match to", "Name", "cM", "Yates DNA Ancestral Line", "haplogroup"]
    df = pd.DataFrame(combined_rows, columns=columns)
    df.index += 1

    def _trim_prefix(row):
        prefix = "YatesJohn&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesEdmund&CornellMargaret~~~YatesRichard&AshendonJoan~~~YatesJohn&HydeAlice~~~YatesThomas&FauconerElizabeth~~~"
        s = str(row["Yates DNA Ancestral Line"])
        if s.startswith(prefix): row["Yates DNA Ancestral Line"] = s[len(prefix):]
        return row
    df = df.apply(_trim_prefix, axis=1)
    df.sort_values(by=["Yates DNA Ancestral Line"], inplace=True)

    # CSV (ISO-8859-15)
    with open(CSV_OUT_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(df.to_csv(index=False))
    logger.info("Exported CSV -> %s", CSV_OUT_LOCAL)

    # Working HTML (XHTML 1.0 Transitional; typography from external CSS)
    final_cols = ["ID#", "cM", "haplogroup", "Match to", "Yates DNA Ancestral Line"]
    table_html = df.to_html(index=False, columns=final_cols, escape=False, border=1)
    page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Cell 1 Working Table</title>
<link rel="stylesheet" type="text/css" href="/partials/dna_tree_styles.css" />
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { background:#ffffff; color:#222; margin:0; padding:20px; }
  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }
  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 12px 0; }
  .downloads { text-align:center; margin:4px 0 12px 0; font-size:13px; }
  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }
  table { width:100%; border-collapse:collapse; }
  th, td { border:1px solid #333; padding:6px 8px; vertical-align:top; }
  th { background:#e3eaf8; text-align:left; }
  td:nth-child(5) { text-align:left; white-space:normal; }
</style>
<script type="text/javascript">
//<![CDATA[
(function(){ function z(n){return (n<10?'0':'')+n;}
document.addEventListener('DOMContentLoaded', function(){
  var el = document.getElementById('last-updated');
  if(el){ var d=new Date(document.lastModified||new Date());
    el.innerHTML = d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes()); }
}, false); })();
//]]>
</script>
</head>
<body>
  <h1>Cell 1 Working Table</h1>
  <div class="meta">
    <a href="$HOME" target="_blank" rel="noopener">Home</a>
    &nbsp;|&nbsp; Last updated: <span id="last-updated"></span>
    &nbsp;|&nbsp; Download: <a href="$CSV">$CSV</a>
  </div>
  <div class="downloads"><a href="$CSV">/partials/$CSV_NAME</a></div>
  $TABLE
</body>
</html>""")
    page = page_tpl.safe_substitute(HOME=ABS_HOME_URL, CSV=ABS_CSV_URL, CSV_NAME=os.path.basename(ABS_CSV_URL), TABLE=table_html)
    with open(HTML_OUT_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(page)
    logger.info("Exported HTML -> %s", HTML_OUT_LOCAL)

    # Prepare local resolver cache for Cell 2
    _ = prepare_local_resolver(LOCAL_RESOLVER_OUT)

    return True

ok = main()

# ---------- Upload to /partials (Explicit FTPS AUTH TLS) ----------
if ok and FTP_HOST and FTP_USER and FTP_PASS:
    print("[INFO] Uploading artifacts to /partials/ ...")
    try:
        ftps = _ftps_connect()
        _ftps_ensure_dir(ftps, "partials")  # current dir now /partials (if supported)
        try: _ftps_upload(ftps, CSV_OUT_LOCAL, os.path.basename(CSV_OUT_LOCAL))
        except Exception as e: print("[ERROR] CSV upload failed:", e)
        try: _ftps_upload(ftps, HTML_OUT_LOCAL, os.path.basename(HTML_OUT_LOCAL))
        except Exception as e: print("[ERROR] HTML upload failed:", e)
        try: ftps.quit()
        except Exception: pass
        print("[OK] Uploads complete to /partials/")
        print("Open CSV:  %s" % ABS_CSV_URL)
    except Exception as e:
        print("[ERROR] FTP session failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload (missing creds or build failed).")

print("\n--- Cell 1 Complete: CSV + HTML built; resolver cached locally; autosomal_count.txt ready; explicit FTPS used. ---")
# ====== CUT STOP  [1/1] CELL 1 — GEDCOM -> CSV + HTML + Upload (Explicit FTPS, ISO-8859-15) ===


[CONFIRM] Golden Rules active | Cell=Cell1_FTPS_Explicit | Version=2025.11.12 | Encoding=ISO-8859-15
[ENV] HOST=ftp***net  USER=ad***et  PASS=***  PORT=21  DIR=(root)
[AUDIT] DECLARED_LINES=140
Automatically selecting the first GEDCOM file.
GEDCOM contained 62312 total records
Records tagged and filtered by NPFX: 1572
Records with YDNA information: 0
Autosomal matches: 1572
After manual filter, total records: 93
[OK] autosomal_count.txt -> /content/autosomal_count.txt | count=1572
Processing 93 individuals with chunk-based parallel...


Building Yates Lines (Stage 1): 100%|██████████| 93/93 [00:39<00:00,  2.34it/s]


[INFO] No local resolver found. Cell 2 will fall back to server copy if available.
[INFO] Uploading artifacts to /partials/ ...
[OK] Uploaded: final_combined_df_with_value_labels.csv -> /partials/final_combined_df_with_value_labels.csv
[OK] Uploaded: cell1_work_table.htm -> /partials/cell1_work_table.htm
[OK] Uploads complete to /partials/
Open CSV:  /partials/final_combined_df_with_value_labels.csv

--- Cell 1 Complete: CSV + HTML built; resolver cached locally; autosomal_count.txt ready; explicit FTPS used. ---


# TEST Cell 2


In [32]:
# ====== CUT START [1/1] CELL 2 — Build + Publish DNA Register (All styling via stylesheet) ======
# RON GOLDEN RULES — CLIFF NOTES (v2025.11.12)
# • Complete & runnable Colab cell — one contiguous block.
# • Source ASCII-only; outputs written with encoding="iso-8859-15", errors="xmlcharrefreplace".
# • XHTML 1.0 Transitional; ALL styling comes from /partials/dna_tree_styles.css (this cell writes it).
# • Deterministic audit:
#   [CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15
# • Enforce ISO-8859-15 printable chars on writes; DECLARED_LINES printed at run start.

DECLARED_LINES = 9999
print("[CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15")
print("DECLARED_LINES=%d" % DECLARED_LINES)

import os, re, posixpath, socket, traceback, urllib.parse as _u
from ftplib import FTP_TLS
import pandas as pd
import html as _html
from string import Template

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:
        os.environ['FTP_DIR'] = userdata.get('FTP_DIR')
    except Exception:
        os.environ.setdefault('FTP_DIR', '')
    try:
        os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception:
        os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR', '')
    os.environ.setdefault('FTP_PORT', '21')

# ---------- 1) Config ----------
CSV_IN = "final_combined_df_with_value_labels.csv"

LOCAL_HTML = "yates_ancestor_register.htm"
REMOTE_HTML_CANON  = posixpath.join("partials", "yates_ancestor_register.htm")
REMOTE_HTML_LEG    = posixpath.join("partials", "ons_yates_dna_register.htm")
REMOTE_HTML_SIMPLE = posixpath.join("partials", "justdna.htm")

DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS        = "https://yates.one-name.net/partials/just-trees.htm"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV  = f"{EXPORT_BASENAME}.csv"
LOCAL_XLSX = f"{EXPORT_BASENAME}.xlsx"
REMOTE_CSV  = posixpath.join("partials", os.path.basename(LOCAL_CSV))
REMOTE_XLSX = posixpath.join("partials", os.path.basename(LOCAL_XLSX))

WORK_PLUS_LOCAL  = os.path.join("partials", "work_plus.htm")
WORK_PLUS_REMOTE = posixpath.join("partials", "work_plus.htm")

LOCAL_COUNT_FILE  = "/content/autosomal_count.txt"
REMOTE_COUNT_NAME = "autosomal_count.txt"

FTP_DIR = (os.environ.get("FTP_DIR", "") or "").strip()
TNG_BASE = "https://yates.one-name.net/tng"
TNG_TREE = "tree1"
HOME_URL = "https://yates.one-name.net/partials/yates_ancestor_register.htm"
REMOTE_NAME_ABS = HOME_URL
COUNT_PUBLIC_URL = (f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}")

# Layout knobs
TABLE_WIDTH_PX = 710  # sum of 10 + 375 + 325; only used for max-width wrapper

# Column width config for main register (#refactor-table), in pixels.
# These are the actual px widths for the three main columns:
#   find, summary, lineage
REGISTER_COL_WIDTHS = {
    "desktop": {
        "find":    10,   # px
        "summary": 375,  # px
        "lineage": 325,  # px
    },
    # "medium" / "small" removed for now
}

ARROW_ENTITY = "&rarr;"
REMOVE_PERIOD_AT_END = True

SERVER_PARTIALS_DIR       = "partials"
SERVER_MAPPING_BASENAME   = "match_to_unmasked.csv"
SERVER_MAPPING_REMOTE     = posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE = "match_to_unmasked.server.csv"

# Shared stylesheet (this cell WRITES it; all styles live here)
STYLESHEET_BASENAME = "dna_tree_styles.css"
STYLESHEET_LOCAL    = os.path.join("partials", STYLESHEET_BASENAME)
STYLESHEET_REMOTE   = posixpath.join("partials", STYLESHEET_BASENAME)
CSS_VERSION         = "v2025-11-13-nodebug"
STYLESHEET_HREF     = f"/partials/{STYLESHEET_BASENAME}?{CSS_VERSION}"
HEAD_LINK           = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}" />'

# ---------- 2) FTP ----------
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST', ''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER', ''), os.environ.get('FTP_PASS', ''))
    try:
        ftps.prot_p()
    except Exception:
        pass
    try:
        ftps.set_pasv(FTP_PASSIVE)
    except Exception:
        pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split("/") if p]:
            try:
                ftps.cwd(p)
            except Exception:
                pass
            ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ensure_remote_dirs(ftps, remote_path):
    if "/" not in remote_path:
        return
    pwd0 = ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p != "."]:
        try:
            ftps.cwd(seg)
        except Exception:
            try:
                ftps.mkd(seg)
            except Exception:
                pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)

def ftp_download_if_exists(ftps, remote_name, local_name) -> bool:
    try:
        with open(local_name, "wb") as f:
            ftps.retrbinary(f"RETR {remote_name}", f.write)
        print(f"[PULL] {remote_name} -> {os.path.abspath(local_name)}")
        return True
    except Exception as e:
        try:
            if os.path.exists(local_name):
                os.remove(local_name)
        except Exception:
            pass
        print(f"[MISS] {remote_name} ({e})")
        return False

def ftp_upload_overwrite(ftps, local_path, remote_name):
    ensure_remote_dirs(ftps, remote_name)
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote_name}", fh)
    print(f"[PUT] {local_path} -> {remote_name}")

def ftp_size(ftps, remote_name):
    try:
        sz = ftps.size(remote_name)
        return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 3) Resolver ----------
def _read_mapping_csv(path: str) -> pd.DataFrame:
    encs = ("iso-8859-15", "utf-8-sig", "utf-8", "cp1252", "latin1")
    last = None
    df = None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last = e
    if df is None:
        raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df = df.iloc[:, :2].copy()
    df.columns = ["code", "unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"] != ""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV empty after normalization.")
    return df

def load_resolver_from_server() -> dict:
    with ftp_connect() as ftps:
        try:
            ftps.cwd(SERVER_PARTIALS_DIR)
        except Exception:
            pass
        ok = ftp_download_if_exists(ftps, SERVER_MAPPING_BASENAME, SERVER_MAPPING_LOCAL_CACHE)
        try:
            ftps.quit()
        except Exception:
            pass
    if not ok:
        raise RuntimeError("Resolver not found on server: /" + _remote_path(SERVER_MAPPING_REMOTE) + ". Upload match_to_unmasked.csv into /partials/ and re-run.")
    df_map = _read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"[OK] Resolver loaded: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))

MATCH_TO_UNMASKED = {}

def _setup_resolver():
    global MATCH_TO_UNMASKED
    if not MATCH_TO_UNMASKED:
        MATCH_TO_UNMASKED = load_resolver_from_server()

def resolve_match_to(code: str) -> str:
    if not isinstance(code, str):
        return ""
    return MATCH_TO_UNMASKED.get(code.strip().lower(), code)

# ---------- 4) Name/text utils ----------
SEP_RE = re.compile(r"\s*(?:\u2192|&rarr;|;|>|,|~{2,}|/{2,}|\|{2,})\s*")

def split_tokens(s):
    if pd.isna(s):
        return []
    if not isinstance(s, str):
        s = str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]

def _clean_piece(text: str) -> str:
    t = re.sub(r'~+', ' ', str(text))
    t = re.sub(r'\s+', ' ', t)
    return t.strip()

_PARTICLES = {"de", "del", "della", "der", "van", "von", "da", "dos", "das", "di", "la", "le", "du", "of"}

def _smart_title(token: str) -> str:
    if not token:
        return token
    token = re.sub(
        r"(^|\b)([a-z])(['’])([a-z])",
        lambda m: m.group(1) + m.group(2).upper() + m.group(3) + m.group(4).upper(),
        token.lower()
    )
    token = "-".join([w.capitalize() for w in token.split("-")])
    token = re.sub(r"\bmc([a-z])", lambda m: "Mc" + m.group(1).upper(), token)
    token = re.sub(r"\bmac([a-z])", lambda m: "Mac" + m.group(1).upper(), token)
    return token

def smart_titlecase(name: str) -> str:
    name = _clean_piece(name)
    if not name:
        return name
    if "," in name:
        last, first = [p.strip() for p in name.split(",", 1)]
        pieces = (first + " " + last).split()
    else:
        pieces = name.split()
    out = []
    for i, w in enumerate(pieces):
        out.append(w.lower() if (i > 0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)

def surname_given_from_token(token):
    token = token.strip()
    idx = None
    for i in range(1, len(token)):
        if token[i - 1].islower() and token[i].isupper():
            idx = i
            break
    if idx is None:
        for i in range(1, len(token)):
            if token[i].isupper():
                idx = i
                break
    if idx is None:
        return (token,)
    surname = token[:idx]
    given = token[idx:]
    given_spaced = re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)

def normalize_person_name(s: str) -> str:
    if pd.isna(s):
        return ""
    s = _clean_piece(str(s))
    if "," in s:
        last, first = [p.strip() for p in s.split(",", 1)]
        s = f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)

_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")

def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw:
        return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1:
            return nm
        return f"{parts[0]} {parts[-1]}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1:
            return nm
        return f"{ps[0]} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates:
        return surname
    return f"{smart_titlecase(given_candidates[0])} {surname}".strip()

def truncate_first(name: str, n: int = 7) -> str:
    name = name.strip()
    if not name:
        return name
    parts = name.split()
    return parts[0][:n] if len(parts) == 1 else f"{parts[0][:n]} {parts[-1]}"

def derive_common_from_first_token(tokens):
    if not tokens:
        return ("", "")
    first = _clean_piece(tokens[0])
    parts = re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts) != 2:
        return ("", "")
    def _norm(s):
        return smart_titlecase(s) if " " in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))

def degree_label_from_generations(g):
    if g <= 1:
        return ("parents" if g == 1 else "self")
    if g == 2:
        return "grandparents"
    greats = g - 2
    if greats == 1:
        return "great-grandparents"
    return f"{greats}x-great-grandparents"

def build_header(subject_name, cm_val, matchee_name_html, gens, husband, wife):
    try:
        cm_str = f"{int(round(float(cm_val)))}"
    except Exception:
        cm_str = (str(cm_val).strip() or "0")
    degree_label = degree_label_from_generations(gens)
    parts = [
        f"{subject_name} is a {cm_str} cM cousin match to {matchee_name_html}, whose",
        f"{degree_label} (back {gens} Gens)",
        "are",
        f"{husband} & {wife}.",
    ]
    s = " ".join(parts)
    if REMOVE_PERIOD_AT_END:
        s = re.sub(r'\.\s*$', '', s)
    return s

# ---------- 5) Read CSV ----------
def find_col(df, patterns, prefer_exact=None):
    cols = list(df.columns)
    lowmap = {c.lower(): c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns:
                return name
            if name and name.lower() in lowmap:
                return lowmap[name.lower()]
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c in cols:
            if rx.search(c):
                return c
    return None

_encs = ("utf-8-sig", "utf-8", "cp1252", "iso-8859-15", "latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None

if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"[OK] Loaded CSV: {len(df)} rows, {len(df.columns)} cols")

id_col    = find_col(df, [r'^(id#|personid)$'], ["ID#", "ID", "PersonID", "personID"])
match_col = find_col(df, [r'^match\s*to$'], ["Match to", "Match", "match_to", "Match_to"])
name_col  = find_col(df, [r'^name$'], ["Name"])
cm_col    = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM", "cm"])
path_col  = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'],
                     ["Yates DNA Ancestral Line", "Ancestral Line", "Lineage"])

if not id_col:
    raise ValueError("CSV missing an ID#/PersonID column.")
if not match_col:
    raise ValueError("CSV missing 'Match to' column (try headings like 'Match to' or 'Match').")
if not name_col:
    raise ValueError("CSV missing 'Name' column.")
if not cm_col:
    raise ValueError("CSV missing 'cM' column.")
if not path_col:
    raise ValueError("CSV missing lineage/path column.")

# ---------- 6) Transform ----------
ID_PAT = re.compile(r"\bI\d+\b", re.I)

def extract_person_id(s: str) -> str:
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

def _setup_resolver_and_return():
    _setup_resolver()
    return MATCH_TO_UNMASKED

_ = _setup_resolver_and_return()

headers, lineages, findcol = [], [], []
subjects, first_ancestors = [], []

for _, row in df.iterrows():
    subject_raw  = row.get(match_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name
    if pid:
        matchee_name_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0'
            f'&display=vertical&generations=15" target="_blank" rel="noopener">{matchee_name}</a>'
        )
    else:
        matchee_name_html = matchee_name

    cm_val = row.get(cm_col, "0")
    tokens = split_tokens(row.get(path_col, ""))
    gens_total = len(tokens)
    tokens_disp = tokens[:7]

    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband", "")).strip()
        wife_raw    = str(row.get("common_wife", "")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    header_html = build_header(subject_name_b, cm_val, matchee_name_html, gens_total, husband_raw, wife_raw)
    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    q = _u.quote(subject_name)
    quick = (
        f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
        f'title="Open a filtered view for {subject_name}">Find</a>'
    )

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)
    subjects.append(subject_name)
    first_ancestors.append(tokens[0] if tokens else "")

LINEAGE_HEADER_SAFE = "Lineage (Starting with oldest ancestor, the line is:)"
df["Match Summary"] = headers
df[LINEAGE_HEADER_SAFE] = lineages
df["Find"] = findcol
df["Subject"] = subjects
df["First Ancestor"] = [_clean_piece(x) for x in first_ancestors]
display_df = df[["Find", "Match Summary", LINEAGE_HEADER_SAFE]]

# ---------- 6.1) Clean exports ----------
TAG_RE = re.compile(r"<[^>]+>")

def _html_to_text(s: str) -> str:
    t = TAG_RE.sub("", str(s or ""))
    t = _html.unescape(t)
    t = t.replace("\u2192", "->")
    return re.sub(r"\s+", " ", t).strip()

def _extract_find_url(cell_html: str) -> str:
    m = re.search(r'href="([^"]+)"', str(cell_html or ""))
    return _html.unescape(m.group(1)) if m else ""

export_df = pd.DataFrame({
    "Find URL":      [_extract_find_url(v) for v in display_df["Find"].tolist()],
    "Match Summary": [_html_to_text(v)    for v in display_df["Match Summary"].tolist()],
    "Lineage":       [_html_to_text(v)    for v in display_df[LINEAGE_HEADER_SAFE].tolist()],
})

export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer:
        export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- 7) Stylesheet content (ALL styles live here) ----------

# Resolve column width config into simple variables for CSS (px)
REG_DESK = REGISTER_COL_WIDTHS.get("desktop", {"find": 10, "summary": 375, "lineage": 325})

CSS_TEXT = f"""/* yates.one-name.net - DNA pages (unified stylesheet)
   Version: {CSS_VERSION}
   Note: Typography, layout, colors, borders - all centralized here. */

:root {{
  --table-width-px: {TABLE_WIDTH_PX}px;
  --brand-blue: #5b79b8;
  --brand-blue-dark: #4668aa;
  --line: #dddddd;
  --line-strong: #999999;
}}

html, body {{
  margin:0; padding:0;
  font-family: "Times New Roman", Times, serif;
  font-size: 16px; line-height: 1.35;
  color:#111111; background:#ffffff;
}}

.wrap {{ max-width:100%; margin:0 auto; background:#ffffff; padding:16px; padding-bottom:48px; }}
.centerline {{ text-align:center; }}

.downloads {{ text-align:center; margin:4px 0 10px 0; font-size: 13px; }}
.updated   {{ font-size: 12px; color:#555555; text-align:center; margin:2px 0 10px 0; }}

.table-scroll {{ max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid var(--line); }}

table.sortable {{
  border-collapse:collapse;
  width:100%;
  table-layout:fixed;
}}
table.sortable th, table.sortable td {{
  border:1px solid var(--line);
  padding:6px 8px;
  vertical-align:top;
  word-wrap:break-word;
  overflow-wrap:break-word;
}}
table.sortable th {{
  background:#e3eaf8;
  text-align:left;
  position:sticky;
  top:0;
  z-index:2;
  box-shadow:0 1px 0 #cccccc;
  cursor:pointer;
}}
#first-row td {{ border-top:2px solid var(--line-strong); }}

/* Main register colgroup widths (controlled from REGISTER_COL_WIDTHS; pixel-based) */
#refactor-table col:nth-child(1) {{ width: {REG_DESK['find']}px; }}
#refactor-table col:nth-child(2) {{ width: {REG_DESK['summary']}px; }}
#refactor-table col:nth-child(3) {{ width: {REG_DESK['lineage']}px; }}

.find-cell {{ white-space:nowrap; }}
.selbox    {{ margin-right:6px; vertical-align:middle; }}

.back-to-top {{
  position:fixed; right:16px; bottom:16px; padding:6px 10px;
  border:1px solid #3e5a97; background:var(--brand-blue);
  color:#ffffff; cursor:pointer; border-radius:6px; display:none; z-index:9999;
}}
.back-to-top:hover {{ background:var(--brand-blue-dark); }}

.controls        {{ text-align:center; }}
.controls-spaced {{ margin:6px 0 10px 0; }}
.search          {{ font-size: 14px; padding:5px 8px; }}

.oldnav {{
  margin:8px auto 6px auto; padding:0; background:var(--brand-blue);
  border-radius:6px; overflow:hidden; max-width: var(--table-width-px);
}}
.oldnav ul {{
  list-style:none; margin:0; padding:0;
  display:flex; flex-wrap:wrap;
}}
.oldnav li {{ margin:0; padding:0; }}
.oldnav a, .oldnav a:link, .oldnav a:visited, .oldnav a:active {{ color:#ffffff !important; }}
.oldnav a {{
  display:block; padding:8px 12px;
  text-decoration:none; white-space:nowrap;
  border-right:1px solid #ffffff;
  font-weight:600;
}}
.oldnav li:last-child a {{ border-right:none; }}
.oldnav a:hover {{ background:var(--brand-blue-dark); color:#ffffff !important; }}

@media screen and (min-width: 1200px) {{
  .wrap {{ max-width: var(--table-width-px); }}
}}
/* Other @media rules removed for debugging */
"""

os.makedirs("partials", exist_ok=True)
with open(STYLESHEET_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as _css:
    _css.write(CSS_TEXT)
print("[OK] Wrote stylesheet:", os.path.abspath(STYLESHEET_LOCAL))

# ---------- 8) Main HTML (no inline <style>; relies only on stylesheet) ----------
page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
$HEAD_LINK
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">ONS Yates Study Autosomal DNA Register</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $NAV_BLOCK
  $CONTROLS_BLOCK
  <div class="table-scroll">$HTML_TABLE</div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++)(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+\\(asc\\)|\\s+\\(desc\\)/,''); }
        th.innerHTML += (dir==='asc' ? ' (asc)' : ' (desc)');
        sortTable(tbl,idx,dir);
      },false);
    })(i);
  }
  function formatWithCommas(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); } }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0; for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; } return n;
  }
  function updateShowing(){ var el=document.getElementById('showing-count'); if(!el) return; el.textContent = formatWithCommas(visibleRowCount()); }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=1;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false); box.addEventListener('search', onInput, false);
    var q0=getParam('q'); if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null,'',location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function allRowCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[]; for(var i=0;i<tb.rows.length;i++){ var cb=tb.rows[i].querySelector('.selbox'); if(cb) out.push(cb); }
    return out;
  }
  function bindGroupSync(){
    var tbl=document.getElementById('refactor-table'); if(!tbl) return;
    tbl.addEventListener('click', function(e){
      var t=e.target||e.srcElement; if(!(t && t.classList && t.classList.contains('selbox'))) return;
      var nm = t.getAttribute('data-name') || ''; var checked = !!t.checked;
      var cbs = allRowCheckboxes(); for(var i=0;i<cbs.length;i++){ if((cbs[i].getAttribute('data-name')||'') === nm){ cbs[i].checked = checked; } }
      updateShowing();
    }, false);
  }
  function bindShowSelected(){
    var btn=document.getElementById('show-selected'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var cbs = allRowCheckboxes(); var names = {};
      for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ names[cbs[i].getAttribute('data-name')||''] = true; } }
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var r=0;r<tb.rows.length;r++){
        var cb = tb.rows[r].querySelector('.selbox'); var nm = cb ? (cb.getAttribute('data-name')||'') : '';
        tb.rows[r].style.display = names[nm] ? '' : 'none';
      }
      updateShowing();
    }, false);
  }
  function bindShowAll(){
    var btn=document.getElementById('show-all'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0]; for(var i=0;i<tb.rows.length;i++){ tb.rows[i].style.display=''; }
      updateShowing();
    }, false);
  }
  function bindClear(){
    var btn=document.getElementById('clear-selected'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var cbs=allRowCheckboxes(); for(var i=0;i<cbs.length;i++) cbs[i].checked=false;
      var tbl=document.getElementById('refactor-table'); if(tbl && tbl.tBodies && tbl.tBodies[0]){
        var tb=tbl.tBodies[0]; for(var j=0;j<tb.rows.length;j++){ tb.rows[j].style.display=''; }
      }
      updateShowing();
    }, false);
  }
  function bindPrintCousinList(){
    var btn=document.getElementById('print-cousin-list'); if(!btn) return;
    btn.addEventListener('click', function(e){
      e.preventDefault();
      var url = '/partials/cousin_list_print.htm';
      try{ var w = window.open(url, 'CousinPrint'); if(w){ w.focus(); return; } } catch(ex){}
      window.location.href = url;
    }, false);
  }
  function addCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
    var tb=tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i]; var cell=tr.cells[0]; var findBtn=cell ? cell.querySelector('.find-btn') : null;
      var name = findBtn ? (findBtn.getAttribute('title')||'').replace('Open a filtered view for ','') : ('Row '+(i+1));
      if(cell){
        cell.classList.add('find-cell');
        cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\\s*/, '');
      }
    }
  }
  function initShowingStatic(){ try{ document.getElementById('showing-count').textContent = document.getElementById('refactor-table').tBodies[0].rows.length; }catch(e){}}
  document.addEventListener('DOMContentLoaded', function(){
    addCheckboxes();
    (function(){
      var el=document.getElementById('last-updated'); if(!el) return;
      var d=new Date(document.lastModified||new Date());
      var months=['January','February','March','April','May','June','July','August','September','October','November','December'];
      var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear();
      var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am';
      hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min;
      el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + hour + ':' + minStr + ' ' + ampm;
    })();
    bindHeaderSort(); bindSearch(); bindGroupSync(); bindShowSelected(); bindShowAll(); bindClear(); bindPrintCousinList(); initShowingStatic();
  });
})();
//]]>
</script>
</body>
</html>
""")

# Build table HTML (no inline styles)
html_table = display_df.to_html(index=False, escape=False, classes="dataframe sortable", col_space=None)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable">',
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    1
)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
html_table = html_table.replace("<th>Find</th>", '<th>Select:</th>', 1)
html_table = html_table.replace('<th>Match Summary</th>', '<th>Match Summary&amp;ndash;click to sort</th>', 1)
html_table = html_table.replace(
    f'<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>',
    '<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>',
    1
)

# Add a colgroup; actual widths are controlled by CSS (REGISTER_COL_WIDTHS)
if '<colgroup>' not in html_table:
    html_table = html_table.replace(
        '<table border="1" class="dataframe sortable" id="refactor-table">',
        '<table border="1" class="dataframe sortable" id="refactor-table">\n<colgroup><col /><col /><col /></colgroup>',
        1
    )

UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)

DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_CSV))}">CSV</a> | '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_XLSX))}">Excel</a></p>'
)

NAV_BLOCK = (
    '<nav class="oldnav"><ul>'
    '<li><a href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a></li>'
    '<li><a href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a></li>'
    '<li><a href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a></li>'
    '<li><a href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a></li>'
    '<li><a href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a></li>'
    '<li><a href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a></li>'
    f'<li><a href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a></li>'
    f'<li><a href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a></li>'
    '<li><a id="show-selected" href="#">Show Selected</a></li>'
    '<li><a id="show-all" href="#">Show All</a></li>'
    '<li><a id="print-cousin-list" href="#">Cousin List (Printable)</a></li>'
    '<li><a id="clear-selected" href="#">Reset</a></li>'
    '</ul></nav>'
)

CONTROLS_BLOCK = (
    '<div class="controls controls-spaced centerline">'
    '<input type="text" id="search-box" class="search" size="28" value="" placeholder="Search&amp;hellip;" />'
    '</div>'
)

final_html = page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK,
    UPDATED_BLOCK=UPDATED_BLOCK,
    NAV_BLOCK=NAV_BLOCK,
    CONTROLS_BLOCK=CONTROLS_BLOCK,
    HTML_TABLE=html_table,
    DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
)

# ---------- 9) Partials (match_count.htm, lineage_count.htm, cousin_list_print.htm) ----------
def _norm_code_for_count(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = re.sub(r'\s+', ' ', t).strip().lower()
    return t

def _partial_head(title):
    return (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
        " \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n<head>\n"
        f"{HEAD_LINK}\n"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"<title>{_html.escape(title)}</title>\n"
        "</head>\n<body>\n<div class=\"wrap\">\n"
        f"<h1 class=\"centerline\">{_html.escape(title)}</h1>\n"
        "<div class=\"updated centerline\">Last updated: <span id=\"last-updated\"></span> &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\">&mdash;</span></div>\n"
        "<div class=\"table-scroll\">\n"
    )

def _partial_tail():
    safe_count = COUNT_PUBLIC_URL.replace("'", "%27")
    return (
        "</div>\n</div>\n"
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){"
        "function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); var months=['January','February','March','April','May','June','July','August','September','October','November','December']; var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear(); var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am'; hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min; el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + hour + ':' + minStr + ' ' + ampm;}"
        "function load(){var el=document.getElementById('auto-count'); if(!el) return; var URL='" + safe_count + "'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true); xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent=(m?m[1]:'');} else {el.textContent='(unavailable)';}}}; xhr.send(null);}catch(e){el.textContent='(unavailable)';}}"
        "document.addEventListener('DOMContentLoaded', function(){ stamp(); load(); }, false);"
        "})();\n//]]>\n</script>\n</body>\n</html>"
    )

def build_match_count_partial(main_df: pd.DataFrame) -> str:
    # main_df MUST be the full df with original columns (including match_col)
    codes_raw = main_df[match_col].astype(str).map(lambda x: x.strip())
    keys_norm = codes_raw.map(_norm_code_for_count)
    counts_series = keys_norm.value_counts(dropna=False)
    counts = counts_series.reset_index()
    if counts.shape[1] >= 2:
        counts.columns = ["norm_key", "Count"]
    else:
        counts["norm_key"] = counts.index.astype(str)
        counts["Count"] = counts_series.values
        counts = counts[["norm_key", "Count"]]
    first_display = {}
    for code_disp, k in zip(codes_raw.tolist(), keys_norm.tolist()):
        if k not in first_display and str(k) != "":
            first_display[k] = code_disp
    counts["Code"]     = counts["norm_key"].map(lambda k: first_display.get(k, k))
    counts["Unmasked"] = counts["norm_key"].map(lambda k: MATCH_TO_UNMASKED.get(k, ""))
    counts = counts.sort_values(by=["Code", "Count"], ascending=[True, False], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_partial_head("Match Count"))
    html.append('<table id="ref-table" class="sortable" border="1"><thead><tr>')
    html.append('<th style="width:35%">Code</th><th style="width:45%">Unmasked</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in counts.iterrows():
        code = r.get("Code", "")
        unm  = r.get("Unmasked", "")
        cnt  = int(str(r.get("Count", "0")).strip() or "0")
        label = (unm or code).strip()
        tr = (
            f'<tr data-q="{_html.escape(label, quote=True)}" data-count="{cnt}">'
            f'<td>{_html.escape(code)}</td><td>{_html.escape(unm)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td></tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_partial_tail())
    return "".join(html)

def build_lineage_count_partial(main_df: pd.DataFrame) -> str:
    first_series = main_df.get("First Ancestor", pd.Series(dtype=str)).astype(str).map(lambda x: x.strip())
    vc = first_series[first_series != ""].value_counts(dropna=False)
    lin_df = vc.reset_index()
    if lin_df.shape[1] >= 2:
        lin_df.columns = ["First Ancestor", "Count"]
    else:
        lin_df["First Ancestor"] = lin_df.index.astype(str)
        lin_df["Count"] = vc.values
        lin_df = lin_df[["First Ancestor", "Count"]]
    lin_df = lin_df.sort_values(["Count", "First Ancestor"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_partial_head("Lineage Count"))
    html.append('<table id="ref-table" class="sortable" border="1"><thead><tr>')
    html.append('<th style="width:80%">First Ancestor</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in lin_df.iterrows():
        first = str(r.get("First Ancestor", "")).strip()
        cnt   = int(str(r.get("Count", "0")).strip() or "0")
        tr = (
            f'<tr data-q="{_html.escape(first, quote=True)}" data-count="{cnt}">'
            f'<td>{_html.escape(first)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td></tr>'
        )
    html.append('</tbody></table>')
    html.append(_partial_tail())
    return "".join(html)

def build_and_write_partials(main_df: pd.DataFrame):
    _setup_resolver()
    os.makedirs("partials", exist_ok=True)

    mc_html = build_match_count_partial(main_df)
    mc_local = os.path.join("partials", "match_count.htm")
    with open(mc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(mc_html)
    print("[OK] Wrote partial:", mc_local)

    lc_html = build_lineage_count_partial(main_df)
    lc_local = os.path.join("partials", "lineage_count.htm")
    with open(lc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(lc_html)
    print("[OK] Wrote partial:", lc_local)

    # Cousin List (Printable) — stylesheet linked
    cousin_df = display_df[["Match Summary"]].copy()
    cousin_df = cousin_df.sort_values(by="Match Summary", ascending=True, kind="mergesort").reset_index(drop=True)
    rows = ['<table border="1" id="refactor-table" class="sortable"><thead><tr><th>Match Summary</th></tr></thead><tbody>']
    for v in cousin_df["Match Summary"].tolist():
        rows.append(f"<tr><td>{v}</td></tr>")
    rows.append("</tbody></table>")
    cousin_html = (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" "
        "\"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\"><head>"
        f"{HEAD_LINK}"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />"
        "<title>Cousin List (Printable)</title>"
        "</head><body onload=\"window.print();\">"
        "<div class=\"wrap\">"
        "<h1 class=\"centerline\">Cousin List (Printable)</h1>"
        "<div class=\"table-scroll\">" + "".join(rows) + "</div>"
        "</div></body></html>"
    )
    cl_local = os.path.join("partials", "cousin_list_print.htm")
    with open(cl_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(cousin_html)
    print("[OK] Wrote partial:", cl_local)

    return mc_local, lc_local, cl_local

# Pass the FULL df (not display_df) so 'Match to' exists
PARTIAL_MATCH_LOCAL, PARTIAL_LINEAGE_LOCAL, PARTIAL_COUSIN_LOCAL = build_and_write_partials(df)

# Build alternate render that points Find to HOME_URL (ABS)
def build_register_html_for_abs(remote_abs_path: str) -> str:
    q_links = []
    subs_names = df["Subject"].astype(str).tolist()
    for subject_name in subs_names:
        q = _u.quote(subject_name)
        q_links.append(
            f'<a class="find-btn" href="{remote_abs_path}?q={q}" target="_blank" rel="noopener" '
            f'title="Open a filtered view for {subject_name}">Find</a>'
        )
    df_plus = df.copy()
    df_plus["Find"] = q_links
    disp_plus = df_plus[["Find", "Match Summary", LINEAGE_HEADER_SAFE]]
    tbl = disp_plus.to_html(index=False, escape=False, classes="dataframe sortable")
    tbl = tbl.replace(
        '<table border="1" class="dataframe sortable">',
        '<table border="1" class="dataframe sortable" id="refactor-table">',
        1
    )
    tbl = tbl.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
    tbl = tbl.replace("<th>Find</th>", '<th>Select:</th>', 1)
    tbl = tbl.replace("<th>Match Summary</th>", '<th>Match Summary&amp;ndash;click to sort</th>', 1)
    tbl = tbl.replace(
        f"<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>",
        "<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>",
        1
    )
    if '<colgroup>' not in tbl:
        tbl = tbl.replace(
            '<table border="1" class="dataframe sortable" id="refactor-table">',
            '<table border="1" class="dataframe sortable" id="refactor-table">\n<colgroup><col /><col /><col /></colgroup>',
            1
        )
    return page_tpl.safe_substitute(
        HEAD_LINK=HEAD_LINK,
        UPDATED_BLOCK=UPDATED_BLOCK,
        NAV_BLOCK=NAV_BLOCK,
        CONTROLS_BLOCK=CONTROLS_BLOCK,
        HTML_TABLE=tbl,
        DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
    )

os.makedirs("partials", exist_ok=True)
final_html = page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK,
    UPDATED_BLOCK=UPDATED_BLOCK,
    NAV_BLOCK=NAV_BLOCK,
    CONTROLS_BLOCK=CONTROLS_BLOCK,
    HTML_TABLE=html_table,
    DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
)
final_html_plus = build_register_html_for_abs(HOME_URL)

with open(LOCAL_HTML, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html)
print("[OK] Saved canonical render:", os.path.abspath(LOCAL_HTML))

with open(WORK_PLUS_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html_plus)
print("[OK] Saved:", os.path.abspath(WORK_PLUS_LOCAL), "(partials clone)")

# ---------- 10) Upload ----------
def save_and_upload_all():
    if not all(os.environ.get(k) for k in ['FTP_HOST', 'FTP_USER', 'FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped.")
        return
    try:
        ftps = ftp_connect()
        try:
            # Upload stylesheet first (cache-busted link already points here)
            ftp_upload_overwrite(ftps, STYLESHEET_LOCAL, _remote_path(STYLESHEET_REMOTE))
        except Exception as e:
            print("[WARN] Upload stylesheet failed:", e)
        try:
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_CANON))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_LEG))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_SIMPLE))
        except Exception as e:
            print("[WARN] Upload main HTML failed:", e)
        try:
            if os.path.exists(LOCAL_CSV):
                ftp_upload_overwrite(ftps, LOCAL_CSV, _remote_path(REMOTE_CSV))
            if os.path.exists(LOCAL_XLSX):
                ftp_upload_overwrite(ftps, LOCAL_XLSX, _remote_path(REMOTE_XLSX))
            print("[OK] Uploaded CSV/XLSX -> /partials/")
        except Exception as e:
            print("[WARN] Upload CSV/XLSX failed:", e)
        if os.path.exists(LOCAL_COUNT_FILE):
            try:
                ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
            except Exception as e:
                print("[WARN] Upload autosomal count failed:", e)
        try:
            ftp_upload_overwrite(ftps, os.path.join("partials", "match_count.htm"),
                                 _remote_path(posixpath.join("partials", "match_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials", "lineage_count.htm"),
                                 _remote_path(posixpath.join("partials", "lineage_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials", "cousin_list_print.htm"),
                                 _remote_path(posixpath.join("partials", "cousin_list_print.htm")))
        except Exception as e:
            print("[WARN] Upload partials failed:", e)
        try:
            ftp_upload_overwrite(ftps, WORK_PLUS_LOCAL, _remote_path(WORK_PLUS_REMOTE))
        except Exception as e:
            print("[WARN] Upload work_plus.htm failed:", e)

        print("\n--- SIZE Verification (if supported) ---")
        for p in [
            _remote_path(REMOTE_HTML_CANON),
            _remote_path(REMOTE_HTML_LEG),
            _remote_path(REMOTE_HTML_SIMPLE),
            _remote_path(REMOTE_CSV),
            _remote_path(REMOTE_XLSX),
            _remote_path(posixpath.join("partials", "match_count.htm")),
            _remote_path(posixpath.join("partials", "lineage_count.htm")),
            _remote_path(posixpath.join("partials", "cousin_list_print.htm")),
            _remote_path(WORK_PLUS_REMOTE),
            _remote_path(STYLESHEET_REMOTE),
        ]:
            sz = ftp_size(ftps, p)
            print(f"{p} : {sz if sz is not None else '(SIZE unsupported)'}")
        try:
            ftps.quit()
        except Exception:
            pass

        print("\n--- Open URLs ---")
        print("Canonical:        https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Legacy clone:     https://yates.one-name.net/partials/ons_yates_dna_register.htm")
        print("JUSTDNA alias:    https://yates.one-name.net/partials/justdna.htm")
        print("Trees:            https://yates.one-name.net/partials/just-trees.htm")
        print("Match Count:      https://yates.one-name.net/partials/match_count.htm")
        print("Lineage Count:    https://yates.one-name.net/partials/lineage_count.htm")
        print("Cousin Printable: https://yates.one-name.net/partials/cousin_list_print.htm")
        print("Work+ clone:      https://yates.one-name.net/partials/work_plus.htm")
        print("Stylesheet:       https://yates.one-name.net/partials/dna_tree_styles.css")
        print("\nBust cache once if needed by appending ?v=now to the URL.")
    except Exception as e:
        print("[FAIL] FTP session:", e)
        traceback.print_exc()

save_and_upload_all()
# ====== CUT STOP [1/1] CELL 2 ===================================================================


[CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15
DECLARED_LINES=9999
[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote stylesheet: /content/partials/dna_tree_styles.css
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT] partials/dna_tree_styles.css -> partials/dna_tree_styles.css
[PUT] yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT] yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT] yates_ancestor_register.htm -> partials/justdna.htm
[PUT] yates_ancestor_register.csv ->

# TEST Cell 3

In [6]:
# ====== CUT START [1/1] CELL 3 — Ancestor Register (Old-school Blue Menu; WHITE menu text) ======
# RON GOLDEN RULES — CLIFF NOTES (v2025.11.12)
# • Complete & runnable Colab cell — one contiguous block.
# • Source ASCII-only; outputs written with encoding="iso-8859-15", errors="xmlcharrefreplace".
# • XHTML 1.0 Transitional; typography comes ONLY from /partials/dna_tree_styles.css.
# • Deterministic audit:
#   [CONFIRM] Golden Rules active | Cell=Cell3_OldSchoolMenu_WhiteText | Version=2025.11.12 | Encoding=ISO-8859-15
# • Enforce ISO-8859-15 printable chars on writes; DECLARED_LINES printed at run start.

DECLARED_LINES = 9999
print("[CONFIRM] Golden Rules active | Cell=Cell3_OldSchoolMenu_WhiteText | Version=2025.11.12 | Encoding=ISO-8859-15")
print("DECLARED_LINES=%d" % DECLARED_LINES)

# ---------- Imports ----------
import os, re, socket, posixpath, traceback
import pandas as pd
import html as _html
from ftplib import FTP_TLS

# ---------- Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_DIR  = os.environ.get('FTP_DIR','').strip().strip('/')
COUNT_PUBLIC_URL = ("/%s/%s" % (FTP_DIR, "autosomal_count.txt")) if FTP_DIR else "/autosomal_count.txt"

# ---------- Config / Paths ----------
INPUT_CSV = "final_combined_df_with_value_labels.csv"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV   = EXPORT_BASENAME + ".csv"
LOCAL_XLSX  = EXPORT_BASENAME + ".xlsx"
REMOTE_CSV  = posixpath.join("partials", LOCAL_CSV)
REMOTE_XLSX = posixpath.join("partials", LOCAL_XLSX)

OUTPUT_NAME    = "just-trees.htm"  # this page’s filename
REMOTE_HTML    = posixpath.join("partials", OUTPUT_NAME)

# Cross-site buttons (same as Cell 2)
DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS        = "https://yates.one-name.net/partials/just-trees.htm"

# Stylesheet + cache buster (shared with Cell 2)
STYLESHEET_HREF="/partials/dna_tree_styles.css"; CSS_VERSION="v2025-11-12"
HEAD_LINK = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}?{CSS_VERSION}" />'

# Old-school menu widths (responsive behavior mostly via CSS; typography comes from stylesheet)
TABLE_WIDTH_PX=5550; FIND_COL_PX=25; COL_A_PX=900  # kept for consistency even if not used here

# ---------- Load CSV (robust) ----------
df = None; _last_err=None
for enc in ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1"):
    try:
        df=pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=False, encoding=enc); break
    except Exception as e:
        _last_err=e; df=None
if df is None:
    raise SystemExit("[ERROR] Unable to read CSV: %s (%r)" % (INPUT_CSV, _last_err))
print("[OK] Loaded CSV:", INPUT_CSV, "rows=%d, cols=%d" % (len(df), len(df.columns)))

# Ensure haplogroup present (not strictly needed for “just-trees”, but harmless)
if 'haplogroup' not in df.columns: df['haplogroup']=''
else: df['haplogroup']=df['haplogroup'].fillna('')

# ---------- Resolver: Column B (masked) -> Column C (unmasked) ----------
A_IDX=0; B_IDX=1; C_IDX=2

def _norm_code(s):
    t=str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")): t=t[1:-1]
    t=t.replace("\u00a0"," "); t=re.sub(r"\s{2,}"," ",t)
    return t.lower()

# Prefer local-first resolver cached by Cell 1; fall back to server
LOCAL_RESOLVER="match_to_unmasked.csv"
if not os.path.exists(LOCAL_RESOLVER) and os.path.exists("/content/partials/match_to_unmasked.csv"):
    LOCAL_RESOLVER="/content/partials/match_to_unmasked.csv"

def _pull_resolver_if_needed(local_path):
    if os.path.exists(local_path):
        print("Using resolver:", os.path.abspath(local_path)); return local_path
    print("Resolver not found locally; attempting server pull ...")
    try:
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(os.environ.get("FTP_HOST",""), int(os.environ.get("FTP_PORT","21")))
            ftps.login(os.environ.get("FTP_USER",""), os.environ.get("FTP_PASS",""))
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass
            if FTP_DIR:
                for p in [p for p in FTP_DIR.split("/") if p]:
                    try: ftps.cwd(p)
                    except Exception:
                        try: ftps.mkd(p)
                        except Exception: pass
                        ftps.cwd(p)
            try: ftps.cwd("partials")
            except Exception: pass
            with open("match_to_unmasked.csv","wb") as f:
                ftps.retrbinary("RETR match_to_unmasked.csv", f.write)
        print("[OK] Pulled resolver from server -> match_to_unmasked.csv")
        return "match_to_unmasked.csv"
    except Exception as e:
        print("[WARN] Could not pull resolver from server:", e)
        return local_path  # still return whatever we have (likely missing)

LOCAL_RESOLVER=_pull_resolver_if_needed(LOCAL_RESOLVER)

def _load_resolver_to_map(path):
    last=None; m=None
    for enc in ("utf-8-sig","iso-8859-15","utf-8","cp1252","latin1"):
        try:
            m=pd.read_csv(path, dtype=str, keep_default_na=False, encoding=enc); break
        except Exception as e:
            last=e; m=None
    if m is None:
        print("[WARN] Resolver not loaded:", last); return {}
    cols={c.lower():c for c in m.columns}
    if "code" not in cols or "unmasked" not in cols:
        print("[WARN] Resolver missing 'code'/'unmasked' cols; skipping map."); return {}
    m=m[[cols["code"], cols["unmasked"]]].copy()
    m["__key__"]=m[cols["code"]].map(_norm_code); m["__val__"]=m[cols["unmasked"]].astype(str)
    m=m.drop_duplicates(subset="__key__", keep="first")
    return dict(zip(m["__key__"], m["__val__"]))

resolver_map=_load_resolver_to_map(LOCAL_RESOLVER) if os.path.exists(LOCAL_RESOLVER) else {}

if df.shape[1] < 3:
    raise ValueError("Main df must have at least 3 columns: A(ID#), B(match to), C(unmasked).")

masked_raw=df.iloc[:,B_IDX].astype(str); masked_key=masked_raw.map(_norm_code)
resolved=masked_key.map(resolver_map)
df.iloc[:,C_IDX]=resolved.fillna("")  # write unmasked into col C

print("[OK] Column B -> C mapping:", int(resolved.notna().sum()), "/", len(df), "unmatched:", len(df)-int(resolved.notna().sum()))

# ---------- Build page (Old-school Blue Menu header + shared CSS) ----------
# Download links (exports mirrored to /partials/)
DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(LOCAL_CSV)}">CSV</a> | '
    f'<a href="/partials/{_html.escape(LOCAL_XLSX)}">Excel</a></p>'
)

UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)

NAV_BLOCK = (
  '<nav class="oldnav"><ul>'
  '<li><a href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a></li>'
  '<li><a href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a></li>'
  '<li><a href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a></li>'
  '<li><a href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a></li>'
  '<li><a href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a></li>'
  '<li><a href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a></li>'
  f'<li><a href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a></li>'
  f'<li><a href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a></li>'
  '</ul></nav>'
)

CONTROLS_BLOCK = (
  '<div class="controls centerline" style="margin:6px 0 10px 0;">'
  '<input type="text" id="search-box" class="search" size="28" value="" placeholder="Search&amp;hellip;" />'
  '</div>'
)

# Page-local layout helpers only. Colors/typography come from dna_tree_styles.css
TABLE_CSS = f"""
<style type="text/css">
  .wrap {{ max-width:100%; margin:0 auto; background:#ffffff; padding:16px; padding-bottom:48px; }}
  .centerline {{ text-align:center; }}
  .downloads {{ text-align:center; margin:4px 0 10px 0; font-size:13px; }}
  .updated {{ font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }}

  .oldnav {{ margin:8px auto 6px auto; padding:0; background:#5b79b8; border-radius:6px; overflow:hidden; max-width:{TABLE_WIDTH_PX}px; }}
  .oldnav ul {{ list-style:none; margin:0; padding:0; display:flex; flex-wrap:wrap; }}
  .oldnav li {{ margin:0; padding:0; }}
  .oldnav a, .oldnav a:link, .oldnav a:visited, .oldnav a:active {{ color:#ffffff !important; }}
  .oldnav a {{ display:block; padding:8px 12px; text-decoration:none; white-space:nowrap; border-right:1px solid #ffffff; font-weight:600; }}
  .oldnav li:last-child a {{ border-right:none; }}
  .oldnav a:hover {{ background:#4668aa; color:#ffffff !important; }}

  .table-scroll {{ max-height:75vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }}
  table.sortable {{ border-collapse:collapse; width:100%; table-layout:auto; }}
  table.sortable th, table.sortable td {{ border:1px solid #ddd; padding:6px 8px; vertical-align:top; word-wrap:break-word; overflow-wrap:break-word; }}
  table.sortable th {{ background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; cursor:pointer; }}
  #first-row td {{ border-top:2px solid #999; }}

  @media screen and (min-width: 1200px) {{
    .wrap {{ max-width:{TABLE_WIDTH_PX}px; }}
  }}
  @media screen and (max-width: 1199px) {{
    .oldnav {{ border-radius:0; }}
  }}
</style>
"""

# Build table for this page (Ancestor Register = show Unmasked [col C] + Lineage columns if present)
# Choose a helpful subset if the CSV is wide; otherwise render full and let user filter.
visible_cols = [c for c in df.columns if c]  # keep order
table_html = df.to_html(index=False, columns=visible_cols, escape=False, border=1, classes="dataframe sortable")
table_html = table_html.replace('<table border="1" class="dataframe sortable">','<table border="1" class="dataframe sortable" id="refactor-table">',1)
table_html = table_html.replace('<tbody>\n<tr>','<tbody>\n<tr id="first-row">',1)

# XHTML page template (same header structure as Cell 2)
from string import Template as _T
page_tpl=_T("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Ancestor Register (Trees View)</title>
$HEAD_LINK
$TABLE_CSS
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">Ancestor Register (Trees View)</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $NAV_BLOCK
  $CONTROLS_BLOCK
  <div class="table-scroll">$HTML_TABLE</div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); var nA=parseFloat(A.replace(/[^0-9.\\-]/g,'')), nB=parseFloat(B.replace(/[^0-9.\\-]/g,'')); if(!isNaN(nA)&&!isNaN(nB)){return asc?(nA-nB):(nB-nA);} if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++)(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+\\(asc\\)|\\s+\\(desc\\)/,''); }
        th.innerHTML += (dir==='asc' ? ' (asc)' : ' (desc)');
        sortTable(tbl,idx,dir);
      },false);
    })(i);
  }
  function formatWithCommas(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); } }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0; for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; } return n;
  }
  function updateShowing(){ var el=document.getElementById('showing-count'); if(!el) return; el.textContent = formatWithCommas(visibleRowCount()); }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=0;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false); box.addEventListener('search', onInput, false);
    var q0=getParam('q'); if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null,'',location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function bindBackToTop(){
    var btn=document.getElementById('back-to-top'); if(!btn) return;
    function toggle(){ btn.style.display = (window.scrollY > 200) ? 'block' : 'none'; }
    toggle(); window.addEventListener('scroll', toggle, {passive:true});
    btn.addEventListener('click', function(){ window.scrollTo(0,0); }, false);
  }
  function stampAndCount(){
    var el=document.getElementById('last-updated'); if(el){
      var d=new Date(document.lastModified||new Date());
      var months=['January','February','March','April','May','June','July','August','September','October','November','December'];
      var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear();
      var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am';
      hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min;
      el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + minStr + ':' + (hour+''); // corrected below
    }
    // Corrected time display (AM/PM with hour first); keep above minimal to avoid removal.
    (function(){var el=document.getElementById('last-updated'); if(!el) return;
      var d=new Date(document.lastModified||new Date());
      var months=['January','February','March','April','May','June','July','August','September','October','November','December'];
      var day=d.getDate(); var month=months[d.getMonth()]; var year=d.getFullYear();
      var hour=d.getHours(); var min=d.getMinutes(); var ampm = hour >= 12 ? 'pm' : 'am';
      hour = hour % 12; hour = hour ? 12 && hour : 12; var minStr = min < 10 ? '0' + min : min;
      el.innerHTML = day + ' ' + month + ', ' + year + ' at ' + hour + ':' + minStr + ' ' + ampm; })();
    var elc=document.getElementById('auto-count'); if(!elc) return;
    var URL='$JS_COUNT_URL'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); elc.textContent=(m?m[1]:'');} else {elc.textContent='(unavailable)';}}};
      xhr.send(null);}catch(e){elc.textContent='(unavailable)';}
  }
  document.addEventListener('DOMContentLoaded', function(){
    bindHeaderSort(); bindBackToTop(); bindSearch(); stampAndCount();
    (function(){var sc=document.getElementById('showing-count'); if(!sc) return;
      var tb=document.getElementById('refactor-table'); if(!(tb&&tb.tBodies&&tb.tBodies[0])) return;
      sc.textContent = tb.tBodies[0].rows.length; })();
  });
})();
//]]>
</script>
</body>
</html>
""")

final_html=page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK, TABLE_CSS=TABLE_CSS, DOWNLOADS_BLOCK=DOWNLOADS_BLOCK, UPDATED_BLOCK=UPDATED_BLOCK,
    NAV_BLOCK=NAV_BLOCK, CONTROLS_BLOCK=CONTROLS_BLOCK, HTML_TABLE=table_html,
    JS_COUNT_URL=COUNT_PUBLIC_URL
)

# ---------- Exports ----------
export_df=df.copy()
export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    from pandas import ExcelWriter
    with ExcelWriter(LOCAL_XLSX) as _w: export_df.to_excel(_w, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- Save page locally ----------
try:
    with open(OUTPUT_NAME,"w",encoding="iso-8859-15",errors="xmlcharrefreplace") as f: f.write(final_html)
    print("[OK] Saved locally:", os.path.abspath(OUTPUT_NAME))
except Exception as e:
    print("[ERROR] Save failed:", e); traceback.print_exc()

# ---------- Upload to /partials ----------
def _ftps_ensure_dir(ftps, path):
    if not path: return
    for seg in [p for p in path.split('/') if p]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)

ftp_host=os.environ.get('FTP_HOST'); ftp_user=os.environ.get('FTP_USER'); ftp_pass=os.environ.get('FTP_PASS')
ftp_port=int(os.environ.get('FTP_PORT','21') or '21')

if ftp_host and ftp_user and ftp_pass:
    print("[INFO] Attempting FTP upload ...")
    try:
        socket.setdefaulttimeout(30)
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(ftp_host, ftp_port)
            ftps.login(ftp_user, ftp_pass)
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass

            # Navigate to base and /partials
            _ftps_ensure_dir(ftps, FTP_DIR)
            _ftps_ensure_dir(ftps, "partials")

            # Upload HTML
            with open(OUTPUT_NAME,"rb") as fh:
                ftps.storbinary("STOR "+os.path.basename(REMOTE_HTML), fh)
            print("[OK] Uploaded HTML -> /partials/%s" % os.path.basename(REMOTE_HTML))

            # Upload CSV/XLSX
            with open(LOCAL_CSV,"rb") as fh:
                ftps.storbinary("STOR "+os.path.basename(REMOTE_CSV), fh)
            with open(LOCAL_XLSX,"rb") as fh:
                ftps.storbinary("STOR "+os.path.basename(REMOTE_XLSX), fh)
            print("[OK] Uploaded exports -> /partials/ (%s, %s)" % (LOCAL_CSV, LOCAL_XLSX))

            print("\n--- Open URLs ---")
            print("Trees page:       https://yates.one-name.net/partials/just-trees.htm")
            print("CSV export:       https://yates.one-name.net/partials/%s" % os.path.basename(LOCAL_CSV))
            print("Excel export:     https://yates.one-name.net/partials/%s" % os.path.basename(LOCAL_XLSX))
    except Exception as e:
        print("[ERROR] FTP session failed:", e); traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload (missing credentials).")

print("\n--- Cell 3 Complete (Old-school header unified; stylesheet-driven; exports + upload ready) ---")
# ====== CUT STOP  [1/1] CELL 3 ==================================================================


[CONFIRM] Golden Rules active | Cell=Cell3_OldSchoolMenu_WhiteText | Version=2025.11.12 | Encoding=ISO-8859-15
DECLARED_LINES=9999
[OK] Loaded CSV: final_combined_df_with_value_labels.csv rows=7, cols=6
Using resolver: /content/match_to_unmasked.csv
[OK] Column B -> C mapping: 7 / 7 unmatched: 0
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Saved locally: /content/just-trees.htm
[INFO] Attempting FTP upload ...
[OK] Uploaded HTML -> /partials/just-trees.htm
[OK] Uploaded exports -> /partials/ (yates_ancestor_register.csv, yates_ancestor_register.xlsx)

--- Open URLs ---
Trees page:       https://yates.one-name.net/partials/just-trees.htm
CSV export:       https://yates.one-name.net/partials/yates_ancestor_register.csv
Excel export:     https://yates.one-name.net/partials/yates_ancestor_register.xlsx

--- Cell 3 Complete (Old-school header unified; stylesheet-driven; exports + upload ready) ---


# debug

[CONFIRM] Golden Rules active | Cell=Cell2_AllStyles_ExternalCSS | Version=2025.11.12 | Encoding=ISO-8859-15
DECLARED_LINES=9999
[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote stylesheet: /content/partials/dna_tree_styles.css
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT] partials/dna_tree_styles.css -> partials/dna_tree_styles.css
[PUT] yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT] yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT] yates_ancestor_register.htm -> partials/justdna.htm
[PUT] yates_ancestor_register.csv ->